# Sheath Expansion Simulations Results - Analysis Round 4

Fourth in a series of notebooks analysing the sheath expansions simulations in a manner similar to Bergmann in his 2002 paper. This is the (hopefully) final one which looks at the fits made to individual IVs and then concatenated together after the fact. 

In [41]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['text.usetex'] = False
mpl.rcParams['font.size'] = 14
import xarray as xr
import pandas as pd
import scipy.io as sio
import sys
import os
import glob
import copy
import pathlib as pth
import importlib
import math
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.spice.splopter as spl
import flopter.spice.tdata as td
import flopter.core.ivdata as iv
import flopter.core.fitters as fts
import flopter.core.fitdata as fd
import flopter.core.lputils as lpu
import flopter.core.constants as c
import flopter.spice.inputparser as inp
import flopter.spice.normalise as nrm
import flopter.spice.xrfuncs as xrf
import flopter.spice.decomposition as dc
import flopter.core.colours as col

# Tool for globbing together the run directories you want to combine 

In [2]:
spice_dir = pth.Path('/home/jleland/data/external_big/spice/')
datasets_dir = 'sheath_exp_datasets/10V_cap'
datasets_path = spice_dir / datasets_dir
shexp_datasets_dir = 'shexp_datasets'
shexp_datasets_path = spice_dir / shexp_datasets_dir
os.chdir(spice_dir)

In [3]:
skippable_runs = set([
    'marconi/spice2/sheath_exp_hg/flat_flush_gapfill/alpha_yz_-6.0', # accidental duplicate
    'marconi/spice2/sheath_exp_hg/angled_recessed_as/alpha_yz_-2.0',
    'marconi/spice2/sheath_exp_hg/angled_recessed_as/alpha_yz_-3.0',
    'marconi/spice2/sheath_exp_hg_fflwp/angled_recessed_as/alpha_yz_-2.0',
    'marconi/spice2/sheath_exp_hg_fflwp/angled_recessed_as/alpha_yz_-3.0',
    'marconi/spice2/sheath_exp_hg_fflwp/angled_recessed_as/alpha_yz_-4.0',
    'marconi/spice2/sheath_exp_hg_fflwp/angled_recessed_as/alpha_yz_-5.0',
    'marconi/spice2/sheath_exp_hg_fflwp/angled_recessed_as/alpha_yz_-6.0',
    'marconi/spice2/sheath_exp_hg_fflwp/flat_flush_as/alpha_yz_-2.0',
    'marconi/spice2/sheath_exp_fflwp/angled_recessed_as/alpha_yz_-2.0',
    'marconi/spice2/sheath_exp_fwp/flat_flush_wp-2_as_1_/alpha_yz_-2.0',
    'marconi/spice2/bergmann_bm/flat_flush_lowas/alpha_yz_-1.5',
]) 
skippable_scans = set()
single_sims = set()

In [4]:
non_standard_variables = {'t', 'ProbePot', 'QnPot', 'npartproc', 'Nz', 'Nzmax', 'Ny', 'count', 'Npc', 'snumber', 'nproc'}
desired_variables = (td.DEFAULT_REDUCED_DATASET | non_standard_variables) - {td.OBJECTSCURRENTFLUXE, td.OBJECTSCURRENTFLUXI}

In [5]:

scans_searchstr = [
    '*/*/shexp_shad_fflwp/*',
    '*/*/sheath_exp_fflwp/angled_recessed_as',
    '*/*/sheath_exp_hg/angled_recessed_as',
]
disallowed_angles = []

scans, all_run_dirs = xrf.get_run_dirs(scans_searchstr, skippable_runs=skippable_runs, 
                                       disallowed_angles=disallowed_angles)

[0]: marconi/spice2/sheath_exp_fflwp/angled_recessed_as
	[0,0]: angled_recessed_as/alpha_yz_-10.0
	[0,1]: angled_recessed_as/alpha_yz_-11.0
	[0,2]: angled_recessed_as/alpha_yz_-12.0
	[0,3]: angled_recessed_as/alpha_yz_-15.0
	[0,4]: angled_recessed_as/alpha_yz_-20.0
	[0,5]: angled_recessed_as/alpha_yz_-3.0
	[0,6]: angled_recessed_as/alpha_yz_-30.0
	[0,7]: angled_recessed_as/alpha_yz_-4.0
	[0,8]: angled_recessed_as/alpha_yz_-5.0
	[0,9]: angled_recessed_as/alpha_yz_-6.0
	[0,10]: angled_recessed_as/alpha_yz_-7.0
	[0,11]: angled_recessed_as/alpha_yz_-8.0
	[0,12]: angled_recessed_as/alpha_yz_-9.0
[1]: marconi/spice2/sheath_exp_hg/angled_recessed_as
	[1,0]: angled_recessed_as/alpha_yz_-10.0
	[1,1]: angled_recessed_as/alpha_yz_-12.0
	[1,2]: angled_recessed_as/alpha_yz_-20.0
	[1,3]: angled_recessed_as/alpha_yz_-30.0
	[1,4]: angled_recessed_as/alpha_yz_-5.0
	[1,5]: angled_recessed_as/alpha_yz_-6.0
	[1,6]: angled_recessed_as/alpha_yz_-8.0
[2]: marconi/spice2/shexp_shad_fflwp/angled_recessed_as
	[

In [6]:
spl_hg_path = spice_dir / all_run_dirs[scans[1]][0]
spl_hg_backup_path = list(spl_hg_path.glob('backup_*'))[-1]

spl_wg_path = spice_dir / all_run_dirs[scans[0]][0]
spl_wg_backup_path = list(spl_wg_path.glob('backup_*'))[-1]

print(f"halfgrid ex. - {all_run_dirs[scans[1]][0]}")
print(f"wholegrid ex. - {all_run_dirs[scans[0]][0]}")

halfgrid ex. - marconi/spice2/sheath_exp_hg/angled_recessed_as/alpha_yz_-10.0
wholegrid ex. - marconi/spice2/sheath_exp_fflwp/angled_recessed_as/alpha_yz_-10.0


# Preamble of necessary stuff

This bit creates all teh necessary variables for plotting to commence, including loading the dataset, creating a denormaliser, extracting relevant probe dimensions from an input file and defining a few functions for ease of manipulation. 

In [7]:
splopter_hg =  spl.Splopter(spl_hg_backup_path, reduce=desired_variables, ignore_tzero_fl=True, version=2.14,
                            store_dataframe_fl=True)
splopter_wg =  spl.Splopter(spl_wg_backup_path, reduce=desired_variables, ignore_tzero_fl=True, version=2.14,
                            store_dataframe_fl=True)

parser_hg = inp.InputParser(input_filename=spl_hg_backup_path / 'input.inp')
denormaliser_hg = nrm.Denormaliser(dt=splopter_hg.tdata.dt, input_parser=parser_hg) #, dimensions=3)

parser_wg = inp.InputParser(input_filename=spl_wg_backup_path / 'input.inp')
denormaliser_wg = nrm.Denormaliser(dt=splopter_wg.tdata.dt, input_parser=parser_wg)


Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp_hg/angled_recessed_as/alpha_yz_-10.0/backup_20210111-0904) doesn't seem to be valid.
Continuing anyway.


/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/scipy/io/matlab/mio.py:218: MatReadWarning: Duplicate variable name "npartproc" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp_fflwp/angled_recessed_as/alpha_yz_-10.0/backup_20210211-0426) doesn't seem to be valid.
Continuing anyway.
dx = 1.0, dy = 0.5, dz = 0.5
dx = 1.0, dy = 1.0, dz = 1.0


In [8]:
# Quick look at sheath edge potential

fig, ax = plt.subplots()
ax.plot(splopter_hg.tdata.diagnostics['QnPot'])

In [9]:
probe_desig = {
    'angled_flush': {'theta_p':10.0, 'recession':0.0, 'gap': 1.0e-3},
    'angled_recessed': {'theta_p':10.0, 'recession':1.0e-3, 'gap': 1.0e-3},
    'flat_flush': {'theta_p':0.0, 'recession':0.0, 'gap': 1.0e-3},
    'flat_recessed': {'theta_p':0.0, 'recession':1.0e-3, 'gap': 1.0e-3},
    'flat_flush_gapless': {'theta_p':0.0, 'recession': 0.0, 'gap': 0.0}
}

tableau_palette = ['#377eb8', '#ff7f00', '#4daf4a', 
                   '#f781bf', '#a65628', '#984ea3', 
                   '#999999', '#e41a1c', '#dede00']

cb_palette = ['#4477AA', '#66CCEE', '#228833', '#CCBB44', 
              '#EE6677', '#AA3377', '#BBBBBB']

contrast_palette = ['#FFFFFF', '#DDAA33', '#BB5566', '#004488', '#000000']

vibrant_palette = ['#0077BB', '#33BBEE', '#009988', '#EE7733', 
                   '#CC3311', '#EE3377', '#BBBBBB']

muted_palette = ['#332288', '#88CCEE', '#44AA99', '#117733', '#999933', 
                 '#DDCC77', '#CC6677', '#882255', '#AA4499', '#DDDDDD']

paledark_palette = [
    ['#BBCCEE', '#CCEEFF', '#CCDDAA', '#EEEEBB', '#FFCCCC', '#DDDDDD'],
    ['#222255', '#225555', '#225522', '#666633', '#663333', '#555555'],
]

probe_colour = {probe_label: cb_palette[i] for i, probe_label in enumerate(probe_desig.keys())}
probe_colour

{'angled_flush': '#4477AA',
 'angled_recessed': '#66CCEE',
 'flat_flush': '#228833',
 'flat_recessed': '#CCBB44',
 'flat_flush_gapless': '#EE6677'}

In [10]:
dV = parser_wg.getfloat('mks', 'mks_te') #* c.BOLTZMANN / c.ELEM_CHARGE
T_e = parser_wg.getfloat('mks', 'mks_te')
n_0 = parser_wg.getfloat('mks', 'mks_n0')

L = parser_wg.getfloat('rectangle2', 'yhigh') - parser_wg.getfloat('rectangle2', 'ylow')
g = parser_wg.getfloat('rectangle2', 'ylow') - parser_wg.getfloat('rectangle0', 'yhigh') 
lambda_D = lpu.debye_length(T_e, n_0)
max_theta_p = np.radians(10.0)

print(f"T_e = {T_e} eV \n"
      f"n_0 = {n_0} m^-3 \n"
      f"L = {L} L_d \n"
      f"g = {g} L_d \n"
      f"lambda_D = {lambda_D} \n"
      f"theta_p = {max_theta_p} \n")

T_e = 5.0 eV 
n_0 = 1e+18 m^-3 
L = 300.0 L_d 
g = 60.0 L_d 
lambda_D = 1.6622799720325184e-05 
theta_p = 0.17453292519943295 



## Dataset loading

In [11]:
for ds_path in datasets_path.glob('*'):
    print(ds_path)

/home/jleland/data/external_big/spice/sheath_exp_datasets/10V_cap/se_combined.nc
/home/jleland/data/external_big/spice/sheath_exp_datasets/10V_cap/se_fflwp_ivs.nc
/home/jleland/data/external_big/spice/sheath_exp_datasets/10V_cap/se_fwp_0_ivs.nc
/home/jleland/data/external_big/spice/sheath_exp_datasets/10V_cap/se_fwp_2_ivs.nc
/home/jleland/data/external_big/spice/sheath_exp_datasets/10V_cap/se_hg_fflwp_ivs.nc
/home/jleland/data/external_big/spice/sheath_exp_datasets/10V_cap/se_hg_ivs.nc


In [12]:
# bbm_ds = xr.load_dataset(f'{datasets_dir}/se_bbm_ivs.nc').isel(run=0)
fwp0_ds = xr.load_dataset(f'{datasets_dir}/se_fwp_0_ivs.nc').isel(run=0)
fwp2_ds = xr.load_dataset(f'{datasets_dir}/se_fwp_2_ivs.nc').isel(run=0)
fflwp_ds = xr.load_dataset(f'{datasets_dir}/se_fflwp_ivs.nc').isel(run=0)
hg_fflwp_ds = xr.load_dataset(f'{datasets_dir}/se_hg_fflwp_ivs.nc').isel(run=0)
hg_ds = xr.load_dataset(f'{datasets_dir}/se_hg_ivs.nc').isel(run=0)

combined_ds = xr.load_dataset(f'{datasets_dir}/se_combined.nc')

In [13]:
combined_ds

<xarray.Dataset>
Dimensions:               (probe: 5, run: 5, theta: 14, voltage: 1062)
Coordinates:
  * probe                 (probe) object 'angled_flush' ... 'flat_recessed'
  * voltage               (voltage) float64 -14.98 -14.97 -14.95 ... 0.025 0.05
  * theta                 (theta) float64 2.0 3.0 4.0 5.0 ... 15.0 20.0 30.0
    gap_desc              (run, probe) object '' '' '' '' '' ... '' '' '' '' ''
    gap                   (run, probe) float64 0.001 0.001 0.001 ... 0.001 0.001
    v_f                   (run, probe, theta) float64 nan nan nan ... nan nan
    recession             (run, probe) float64 nan 0.0003 0.0 ... 0.0 nan nan
    theta_p               (run, probe) float64 nan 10.0 0.0 nan ... 0.0 nan nan
    theta_p_rads          (run, probe) float64 nan 0.1745 0.0 ... 0.0 nan nan
    theta_rads            (run, probe, theta) float64 nan nan nan ... nan nan
    recession_descr       (run, probe) object '' 'recessed' 'flush' ... '' ''
    theta_p_descr         (run, probe) object '' 'angled' 'flat' ... '' ''
    recessed_fl           (run, probe) float64 nan 1.0 0.0 nan ... 0.0 nan nan
    angled_fl             (run, probe) float64 nan 1.0 0.0 nan ... 0.0 nan nan
  * run                   (run) object 'hg' 'hg_fflwp' 'fwp_2' 'fwp_0' 'fflwp'
Data variables:
    str_iv_chi2           (run, probe, theta) float64 nan nan nan ... nan nan
    str_iv_reduced_chi2   (run, probe, theta) float64 nan nan nan ... nan nan
    str_iv_I_sat          (run, probe, theta) float64 nan nan nan ... nan nan
    str_iv_d_I_sat        (run, probe, theta) float64 nan nan nan ... nan nan
    str_iv_a              (run, probe, theta) float64 nan nan nan ... nan nan
    str_iv_d_a            (run, probe, theta) float64 nan nan nan ... nan nan
    str_iv_T_e            (run, probe, theta) float64 nan nan nan ... nan nan
    str_iv_d_T_e          (run, probe, theta) float64 nan nan nan ... nan nan
    str_iv_V_f            (run, probe, theta) float64 nan nan nan ... nan nan
    str_iv_d_V_f          (run, probe, theta) float64 nan nan nan ... nan nan
    str_iv_run_dir        (run, probe, theta) object '' '' '' '' ... '' '' '' ''
    str_iv_voltage_min    (run, probe, theta) float64 nan nan nan ... nan nan
    str_iv_voltage_max    (run, probe, theta) float64 nan nan nan ... nan nan
    norm_iv_chi2          (run, probe, theta) float64 nan nan nan ... nan nan
    norm_iv_reduced_chi2  (run, probe, theta) float64 nan nan nan ... nan nan
    norm_iv_I_sat         (run, probe, theta) float64 nan nan nan ... nan nan
    norm_iv_d_I_sat       (run, probe, theta) float64 nan nan nan ... nan nan
    norm_iv_a             (run, probe, theta) float64 nan nan nan ... nan nan
    norm_iv_d_a           (run, probe, theta) float64 nan nan nan ... nan nan
    norm_iv_voltage_min   (run, probe, theta) float64 nan nan nan ... nan nan
    norm_iv_voltage_max   (run, probe, theta) float64 nan nan nan ... nan nan
    ion_chi2              (run, probe, theta) float64 nan nan nan ... nan nan
    ion_reduced_chi2      (run, probe, theta) float64 nan nan nan ... nan nan
    ion_I_sat             (run, probe, theta) float64 nan nan nan ... nan nan
    ion_d_I_sat           (run, probe, theta) float64 nan nan nan ... nan nan
    ion_a                 (run, probe, theta) float64 nan nan nan ... nan nan
    ion_d_a               (run, probe, theta) float64 nan nan nan ... nan nan
    ion_voltage_min       (run, probe, theta) float64 nan nan nan ... nan nan
    ion_voltage_max       (run, probe, theta) float64 nan nan nan ... nan nan
    elec_chi2             (run, probe, theta) float64 nan nan nan ... nan nan
    elec_reduced_chi2     (run, probe, theta) float64 nan nan nan ... nan nan
    elec_T_e              (run, probe, theta) float64 nan nan nan ... nan nan
    elec_d_T_e            (run, probe, theta) float64 nan nan nan ... nan nan
    elec_I_sat            (run, probe, theta) float64 nan nan nan ... nan nan
    elec_d_I_sat          (run, probe, theta) float6

In [263]:
# bbm_ds = xr.load_dataset(f'{datasets_dir}/se_bbm_ivs.nc').isel(run=0)
fwp0_shexp_ds = xr.load_dataset(f'{shexp_datasets_dir}/se_fwp_0_ivs.nc').isel(run=0)
# fwp2_ds = xr.load_dataset(f'{datasets_dir}/se_fwp_2_ivs.nc').isel(run=0)
fflwp_shexp_ds = xr.load_dataset(f'{shexp_datasets_dir}/se_fflwp_ivs.nc').isel(run=0)

combined_shexp_ds = xr.load_dataset(f'{shexp_datasets_dir}/se_combined.nc')
combined_shexp_ds

<xarray.Dataset>
Dimensions:               (probe: 2, run: 2, theta: 15, voltage: 801)
Coordinates:
    gap                   float64 0.001
    theta_p_rads          (probe) float64 0.1745 0.0
    recession_descr       (probe) object 'recessed' 'flush'
    theta_p               (probe) float64 10.0 0.0
    gap_desc              <U1 ''
    recessed_fl           (probe) bool True False
    recession             (probe) float64 0.0003 0.0
    theta_p_descr         (probe) object 'angled' 'flat'
    angled_fl             (probe) bool True False
  * voltage               (voltage) float64 -14.97 -14.95 -14.92 ... 5.0 5.025
  * theta                 (theta) float64 4.0 5.0 6.0 7.0 ... 20.0 25.0 30.0
  * probe                 (probe) object 'angled_recessed' 'flat_flush'
    theta_rads            (run, theta) float64 nan 0.08727 ... 0.4363 0.5236
    v_f                   (run, probe, theta) float64 nan -10.7 ... -1.685
  * run                   (run) object 'fwp_0' 'fflwp'
Data variables:
    str_iv_chi2           (run, probe, theta) float64 nan 155.1 ... 679.9 628.4
    str_iv_reduced_chi2   (run, probe, theta) float64 nan 0.9235 ... 1.349 1.192
    str_iv_I_sat          (run, probe, theta) float64 nan -38.02 ... -89.42
    str_iv_d_I_sat        (run, probe, theta) float64 nan 48.69 ... 0.4116
    str_iv_a              (run, probe, theta) float64 nan 2.061e-13 ... 0.008253
    str_iv_d_a            (run, probe, theta) float64 nan 0.3791 ... 0.0005545
    str_iv_T_e            (run, probe, theta) float64 nan 3.305 ... 0.5167
    str_iv_d_T_e          (run, probe, theta) float64 nan 12.83 ... 0.01452
    str_iv_V_f            (run, probe, theta) float64 nan -2.877 ... -1.644
    str_iv_d_V_f          (run, probe, theta) float64 nan 15.15 ... 0.01071
    str_iv_run_dir        (run, probe, theta) object '' ... 'marconi/spice2/shexp_shad_fflwp/flat_flush_as/alpha_yz_-30.0'
    str_iv_voltage_min    (run, probe, theta) float64 nan -14.97 ... -14.97
    str_iv_voltage_max    (run, probe, theta) float64 nan -10.7 ... -2.275 -1.7
    norm_iv_chi2          (run, probe, theta) float64 nan ... 3.484e+03
    norm_iv_reduced_chi2  (run, probe, theta) float64 nan 255.5 ... 8.816 6.585
    norm_iv_I_sat         (run, probe, theta) float64 nan -42.7 ... -97.27
    norm_iv_d_I_sat       (run, probe, theta) float64 nan 5.264 ... 0.8804
    norm_iv_a             (run, probe, theta) float64 nan 1.58e-21 ... 4.104e-15
    norm_iv_d_a           (run, probe, theta) float64 nan 0.05339 ... 0.001934
    norm_iv_voltage_min   (run, probe, theta) float64 nan -4.275 ... -13.29
    norm_iv_voltage_max   (run, probe, theta) float64 nan -0.025 ... -0.01484
    ion_chi2              (run, probe, theta) float64 nan 145.9 ... 632.5 616.5
    ion_reduced_chi2      (run, probe, theta) float64 nan 0.8842 ... 1.26 1.174
    ion_I_sat             (run, probe, theta) float64 nan 34.52 ... 71.16 90.0
    ion_d_I_sat           (run, probe, theta) float64 nan 0.1183 ... 0.2758
    ion_a                 (run, probe, theta) float64 nan 0.02572 ... 0.01226
    ion_d_a               (run, probe, theta) float64 nan 0.001897 ... 0.0007292
    ion_voltage_min       (run, probe, theta) float64 nan 0.2102 ... 0.1973
    ion_voltage_max       (run, probe, theta) float64 nan 2.973 ... 6.73 6.96
    elec_chi2             (run, probe, theta) float64 nan 2.195e-28 ... 691.4
    elec_reduced_chi2     (run, probe, theta) float64 nan 1.299e-30 ... 1.307
    elec_T_e              (run, probe, theta) float64 nan 4.059 ... 0.5283
    elec_d_T_e            (run, probe, theta) float64 nan 9.206e-31 ... 0.003122
    elec_I_sat            (run, probe, theta) float64 nan ... -2.029e+03
    elec_d_I_sat          (run, probe, theta) float64 nan 3.101e-16 ... 45.91
    elec_voltage_min      (run, probe, theta) float64 nan -14.97 ... -14.97
    elec_voltage_max      (run, probe, theta) float64 nan -10.72 ... -2.275 -1.7
    mf_iv_chi2            (run, probe, theta) float64 nan nan ... 763.6 708.5
    mf

In [15]:
piecewise_ds = xr.load_dataset('piecewise_sheath_exp_ivs.nc')
piecewise_ds

<xarray.Dataset>
Dimensions:         (probe: 1, theta: 7, voltage: 14, y: 557, z: 233)
Coordinates:
  * probe           (probe) object 'flat_flush'
  * z               (z) int64 0 1 2 3 4 5 6 7 ... 226 227 228 229 230 231 232
  * voltage         (voltage) float64 -25.0 -18.0 -12.0 -10.0 ... -2.0 -1.0 0.0
  * y               (y) int64 0 1 2 3 4 5 6 7 ... 550 551 552 553 554 555 556
  * theta           (theta) float64 3.0 5.0 15.0 20.0 12.0 10.0 8.0
Data variables:
    current         (probe, theta, voltage) float64 10.47 10.23 ... -184.3
    d_current       (probe, theta, voltage) float64 3.618 3.592 ... 13.93 16.11
    derr_current    (probe, theta, voltage) float64 0.007561 ... 0.03366
    current_i       (probe, theta, voltage) float64 10.47 10.23 ... 14.75 10.22
    d_current_i     (probe, theta, voltage) float64 3.618 3.592 ... 3.924 3.235
    derr_current_i  (probe, theta, voltage) float64 0.007561 ... 0.006761
    current_e       (probe, theta, voltage) float64 -3.1e-06 ... -194.6
    d_current_e     (probe, theta, voltage) float64 0.001761 0.003295 ... 15.97
    derr_current_e  (probe, theta, voltage) float64 3.68e-06 ... 0.03337
    rho             (probe, theta, voltage, z, y) float64 0.0 0.0 ... 111.5
    pot             (probe, theta, voltage, z, y) float64 0.0 0.0 ... 0.0 0.0
    delta           (probe, theta, voltage) int64 32 26 19 16 12 ... 5 2 3 1 0
    delta_w         (probe, theta, voltage) int64 32 26 19 16 12 ... 5 2 3 1 0
    theta_rads      (probe, theta) float64 -0.05236 -0.08727 ... -0.1745 -0.1396
    theta_p_rads    (probe) object 'flat_flush'
    recession       (probe) object 'flat_flush'

In [16]:
# Loading in slightly older data

lowdens_dir = pth.Path('/home/jleland/data/external/spice/')
fit_data_filename = 'lowdens_fitdata.csv'

fit_df = pd.read_csv(lowdens_dir / fit_data_filename, index_col=0)

flush_df = fit_df.loc[fit_df['probe'] == 'flush']
flush_df = flush_df.loc[flush_df['angle'] <= 30].drop(20).rename(columns={'angle':'theta'})

flush_df['theta_rads'] = np.radians(flush_df['theta'])
flush_df['theta_p'] = 0.0
flush_df['recession'] = 0.0
flush_df['theta_p_rads'] = np.radians(flush_df['theta_p'])

flush_ds = flush_df.to_xarray().swap_dims({'index':'theta'})

# Plot 0 - IV Characteristics and Example Fits

In [17]:
fflwp_ds

<xarray.Dataset>
Dimensions:               (probe: 2, theta: 14, voltage: 602)
Coordinates:
  * probe                 (probe) object 'angled_recessed' 'flat_flush'
    gap_desc              <U1 ''
    gap                   float64 0.001
  * theta                 (theta) float64 2.0 3.0 4.0 5.0 ... 15.0 20.0 30.0
  * voltage               (voltage) float64 -14.97 -14.95 -14.92 ... 0.025 0.05
    v_f                   (probe, theta) float64 nan -4.473 ... -3.278 -2.848
    recession             (probe) float64 0.0003 0.0
    theta_p               (probe) float64 10.0 0.0
    theta_p_rads          (probe) float64 0.1745 0.0
    theta_rads            (probe, theta) float64 nan 0.05236 ... 0.3491 0.5236
    recession_descr       (probe) object 'recessed' 'flush'
    theta_p_descr         (probe) object 'angled' 'flat'
    recessed_fl           (probe) bool True False
    angled_fl             (probe) bool True False
    run                   <U5 'fflwp'
Data variables:
    str_iv_chi2           (probe, theta) float64 nan 6.638e+03 ... 709.9 618.7
    str_iv_reduced_chi2   (probe, theta) float64 nan 12.89 7.215 ... 1.378 1.201
    str_iv_I_sat          (probe, theta) float64 nan -20.24 ... -42.74 -70.22
    str_iv_d_I_sat        (probe, theta) float64 nan 0.3797 ... 0.3601 0.5623
    str_iv_a              (probe, theta) float64 nan 3.188e-17 ... 0.0116
    str_iv_d_a            (probe, theta) float64 nan 0.005496 ... 0.001765
    str_iv_T_e            (probe, theta) float64 nan 1.383 ... 0.9642 1.081
    str_iv_d_T_e          (probe, theta) float64 nan 0.01722 ... 0.01071 0.01478
    str_iv_V_f            (probe, theta) float64 nan -4.585 ... -3.239 -2.862
    str_iv_d_V_f          (probe, theta) float64 nan 0.01138 ... 0.005672
    str_iv_run_dir        (probe, theta) object '' ... 'marconi/spice2/sheath_exp_fflwp/flat_flush_as/alpha_yz_-30.0'
    str_iv_voltage_min    (probe, theta) float64 nan -14.97 ... -14.97 -14.97
    str_iv_voltage_max    (probe, theta) float64 nan -2.025 ... -2.025 -2.025
    norm_iv_chi2          (probe, theta) float64 nan 358.6 479.6 ... 537.3 609.2
    norm_iv_reduced_chi2  (probe, theta) float64 nan 0.8559 ... 1.153 1.259
    norm_iv_I_sat         (probe, theta) float64 nan -16.81 ... -43.71 -68.18
    norm_iv_d_I_sat       (probe, theta) float64 nan 0.0619 ... 0.2498 0.3739
    norm_iv_a             (probe, theta) float64 nan 0.041 ... 0.02003 0.017
    norm_iv_d_a           (probe, theta) float64 nan 0.001072 ... 0.001324
    norm_iv_voltage_min   (probe, theta) float64 nan -10.5 ... -11.7 -12.13
    norm_iv_voltage_max   (probe, theta) float64 nan -0.00249 ... -0.001578
    ion_chi2              (probe, theta) float64 nan 127.8 175.8 ... 264.8 301.4
    ion_reduced_chi2      (probe, theta) float64 nan 0.71 0.9351 ... 1.166 1.23
    ion_I_sat             (probe, theta) float64 nan 16.35 18.25 ... 44.33 70.23
    ion_d_I_sat           (probe, theta) float64 nan 0.07881 ... 0.3273 0.4642
    ion_a                 (probe, theta) float64 nan 0.04974 ... 0.009511
    ion_d_a               (probe, theta) float64 nan 0.002174 ... 0.002269
    ion_voltage_min       (probe, theta) float64 nan 1.002 1.016 ... 1.016 1.001
    ion_voltage_max       (probe, theta) float64 nan 3.605 3.711 ... 4.175 4.373
    elec_chi2             (probe, theta) float64 nan 533.4 476.0 ... 861.8 472.1
    elec_reduced_chi2     (probe, theta) float64 nan 1.273 ... 1.849 0.9755
    elec_T_e              (probe, theta) float64 nan 0.9554 ... 1.004 1.058
    elec_d_T_e            (probe, theta) float64 nan 0.003182 ... 0.003898
    elec_I_sat            (probe, theta) float64 nan -1.748e+03 ... -1.04e+03
    elec_d_I_sat          (probe, theta) float64 nan 30.69 24.22 ... 26.69 12.57
    elec_voltage_min      (probe, theta) float64 nan -14.97 ... -14.97 -14.97
    elec_voltage_max      (probe, theta) float64 nan -4.475 ... -3.3 -2.85
    mf_iv_chi2            (probe, theta) float64 nan 416.2 515.7 ... 763.0 689.1
    mf_iv_reduced_chi2 

In [33]:
# IVs for whole grid

sel_args = {'theta':slice(4,20,4), 'voltage':slice(-14,-2.5)}
probes = {'flat_flush': 'Flush-mounted', 
          'angled_recessed': 'Angled-tip Recessed'}



fig, ax = plt.subplots(1, 2, sharey=True, sharex=True, figsize=[10,6])

markers = ['o', '^', 's']
linestyles = [
    '-',
    '--',
    '-.',
#     ':',
    (0, (1.0, 1.0)),
]
probe_colours = [
    'flat_flush': cb_palette[0],
    'angled_recessed': cb_palette[4],
]

plot_ds = fflwp_shexp_ds
for i, probe in enumerate(probes.keys()):

    probe_ds = plot_ds.sel(probe=probe, **sel_args)
    
    colours = cb_palette[:probe_ds.theta.size]
    for j, theta in enumerate(probe_ds.theta.values):
        probe_angle_ds = probe_ds.sel(theta=theta)
        ax[i].errorbar(probe_angle_ds.voltage, probe_angle_ds.current, ls=linestyles[j],
                       color=contrast_palette[j+1], label=r'{:.2g}$\degree$'.format(theta))
    ax[i].set_title(probes[probe])

#     .where(plot_ds.current <= 0, drop=True)

# ax[0].set_ylim(5e-3, 5e-2)
ax[0].set_xlim(-14, -2.5)


ax[0].set_ylabel(r'$\hat{I}_{tot}$')
ax[0].set_xlabel(r'$\hat{V}$')
ax[1].set_xlabel(r'$\hat{V}$')

for axis in ax:
    axis.axhline(y=0, **c.AX_LINE_DEFAULTS)

ax[1].legend(title=r'$\theta$')
fig.tight_layout()


In [47]:
# IVs for whole grid

sel_args = {'theta':slice(4,20,4), 'voltage':slice(-15,-2.5)}
probes = {'flat_flush': 'Flush-mounted', 
          'angled_recessed': 'Angled-tip Recessed'}



fig, ax = plt.subplots(1, 2, sharey=True, sharex=True, figsize=[10,6])

markers = ['o', '^', 's']
linestyles = [
    '-',
    '--',
    '-.',
#     ':',
    (0, (1.0, 1.0)),
]
probe_colours = {
#     'flat_flush': cb_palette[0],
#     'angled_recessed': cb_palette[4],
    'flat_flush': col.palettes['sb'],
    'angled_recessed': col.palettes['sr'],
}

plot_ds = fflwp_shexp_ds
for i, probe in enumerate(probes.keys()):

    probe_ds = plot_ds.sel(probe=probe, **sel_args)
    
    colours = cb_palette[:probe_ds.theta.size]
    for j, theta in enumerate(probe_ds.theta.values):
        probe_angle_ds = probe_ds.sel(theta=theta)
        ax[i].errorbar(probe_angle_ds.voltage, probe_angle_ds.current, ls=linestyles[j],# alpha=((j+1)/probe_ds.theta.size),
                       color=probe_colours[probe][j+2], label=r'{:.2g}$\degree$'.format(theta))
    ax[i].set_title(probes[probe])

#     .where(plot_ds.current <= 0, drop=True)

# ax[0].set_ylim(5e-3, 5e-2)
ax[0].set_xlim(-15, -2.5)


ax[0].set_ylabel(r'$\hat{I}_{tot}$')
ax[0].set_xlabel(r'$\hat{V}$')
ax[1].set_xlabel(r'$\hat{V}$')

for axis in ax:
    axis.axhline(y=0, **c.AX_LINE_DEFAULTS)

ax[1].legend(title=r'$\theta$')
fig.tight_layout()

In [394]:
# Example plots of ion and straight fits

fig, ax = plt.subplots(1, 2, figsize=[12,6])

plot_ds = combined_shexp_ds.sel(run='fflwp', probe='flat_flush', theta=[6,12,18], voltage=slice(-15,-2.5))
# plot_ds = fflwp_shexp_ds

colours = cb_palette[:probe_ds.theta.size]

i_fitter = fts.IonCurrentSEFitter()
fp_fitter = fts.FullIVFitter()

for j, theta in enumerate(plot_ds.theta.values):
    probe_ds = plot_ds.sel(theta=theta).drop('str_iv_run_dir')
    probe_angle_ds = probe_ds.where(probe_ds.current <= 0.0, drop=True)

    ax[0].errorbar(probe_angle_ds.voltage_cl, probe_angle_ds.current_i, alpha=0.8,
                   color=contrast_palette[j+1])
    
    dummy_probe_angle_ds = probe_angle_ds.isel(voltage=0)
    
    dummy_fit_v = np.linspace(dummy_probe_angle_ds.ion_voltage_max, dummy_probe_angle_ds.ion_voltage_min)
    dummy_fit_y = i_fitter.fit_function(dummy_fit_v, dummy_probe_angle_ds.ion_I_sat.values, 
                                        dummy_probe_angle_ds.ion_a.values)
    ax[0].plot(dummy_fit_v, dummy_fit_y, color=contrast_palette[4], ls=linestyles[j], zorder=5,
               lw=2.0)
#                color=contrast_palette[j+1])    
    
    probe_angle_ds = probe_ds.where(probe_ds.current <= 20.0, drop=True)
    
    ax[1].errorbar(probe_angle_ds.voltage, probe_angle_ds.current, alpha=0.8,
                   color=contrast_palette[j+1], label=r'{:.2g}$\degree$'.format(theta))
    
    dummy_fit_v = np.linspace(dummy_probe_angle_ds.str_iv_voltage_min, 
                              dummy_probe_angle_ds.str_iv_voltage_max)
    dummy_fit_y = fp_fitter.fit_function(dummy_fit_v, 
                                         dummy_probe_angle_ds.str_iv_I_sat.values, 
                                         dummy_probe_angle_ds.str_iv_a.values,
                                         dummy_probe_angle_ds.str_iv_T_e.values,
                                         dummy_probe_angle_ds.str_iv_V_f.values)
    ax[1].plot(dummy_fit_v, dummy_fit_y, color=contrast_palette[4], ls=linestyles[j], zorder=5,
               lw=2.0, label=r'{:.2g}$\degree$ - Fit'.format(theta))   
    
ax[0].set_title('Ion fit')
ax[1].set_title('Straight fit')


ax[0].set_xlim(.0, 6.0)
ax[0].set_ylim(.0, None)
ax[1].set_xlim(-15, -2.5)
ax[1].set_ylim(None, 5.0)
ax[0].set_ylabel(r'$\hat{I}_{i}$')
ax[0].set_xlabel(r'$|\hat{V} - \hat{V}_{fl}|^{3/4}$')

ax[1].set_ylabel(r'$\hat{I}_{tot}$')
ax[1].set_xlabel(r'$\hat{V}$')

for axis in ax:
    axis.axhline(y=0, **c.AX_LINE_DEFAULTS)


fig.tight_layout(rect=(0,0,0.85,1.0))
ax[1].legend(title=r'$\theta$', loc=(1.05, 0.3))


# Plot 1 - Sheath exp parameter $a$ as a function of $\theta$

In [49]:
dummy_theta = np.linspace(0.00001, 45.0, 5000)
flush_calced_a = lpu.calc_sheath_expansion_param(
    T_e,
    n_0,
    L * lambda_D,
    g * lambda_D, 
    np.radians(dummy_theta),
    c_1=0.5,
    c_2=0.6,
)
probe_names = {'flat_flush': 'Flush-mounted', 
               'angled_recessed': 'Angled-tip Recessed'}

In [54]:
# Wholegrid floating sheath exp params, probe comparison

sel_args = {'theta':slice(2,30)}

fig, ax = plt.subplots(figsize=[8,6])

markers = ['o', '^', 's']
colours = (cb_palette[0], cb_palette[4])
mfcs = ['none', None]

plot_ds = hg_fflwp_ds

for i, probe in enumerate(probe_names.keys()):

    probe_ds = plot_ds.sel(probe=probe, **sel_args)

    ax.errorbar(probe_ds.theta, probe_ds.ion_a, 
                yerr=probe_ds.ion_d_a, color=colours[i], mfc='none',
                label=f'{probe_names[probe]}', marker=markers[i], ls='none')
#     ax.errorbar(probe_ds.theta, probe_ds.mf_iv_a, 
#                 yerr=probe_ds.mf_iv_d_a, color=colours[i],
#                 label=f'{probes[probe]}', marker=markers[i], ls='none')
    
calced_a = lpu.calc_sheath_expansion_param(
    T_e, n_0, L * lambda_D, g * lambda_D, np.radians(dummy_theta), 
    c_1=0.9, c_2=0.6,
)
ax.errorbar(dummy_theta, calced_a, label=r'Predicted - Murphy-Sugrue', fmt='-', 
            color=cb_palette[3], linewidth=1.0)

calced_a = lpu.calc_sheath_expansion_param(
    T_e, n_0, L * lambda_D, 0, np.radians(dummy_theta), 
    c_1=0.5, c_2=0.6,
)
ax.errorbar(dummy_theta, calced_a, label=r'Predicted - Bergmann', fmt='--', 
            color=cb_palette[2], linewidth=1.0)
    
ax.set_title(r'$\Delta_y = \Delta_z = 0.5\lambda_D$')
ax.set_ylim(5e-3, 1.5e-1)
ax.set_xlim(-0.5, 30.5)
ax.set_ylabel(r'$a$ [unitless]')
ax.set_xlabel(r'$\theta$ [$^{\circ}$]')

ax.legend()
fig.tight_layout()

In [52]:
# Comparison of hg and wg floating sheath exp params, \w probe comparison
sel_args = {'theta':slice(2,30)}


fig, axes = plt.subplots(1, 2, sharex=True, sharey=True, figsize=[10,6])

markers = ['o', '^', 's']
colours = (cb_palette[0], cb_palette[4])

# title_str = r'$\delta_y = \delta_z = {:.1g}\lambda_D$'
# title_vals = [1.0, 0.5]
# datasets = [fflwp_ds, hg_fflwp_ds]

datasets = [hg_fflwp_ds, fflwp_shexp_ds]
title_str = r'{}elf-shadowing'
title_vals = ['S', 'Non-s']

# datasets = [fwp0_ds, fwp0_shexp_ds]

# plot_ds = fflwp_ds
for j, plot_ds in enumerate(datasets):
    ax = axes[j]
    for i, probe in enumerate(probe_names.keys()):

        probe_ds = plot_ds.sel(probe=probe, **sel_args)

        ax.errorbar(probe_ds.theta, probe_ds.ion_a, 
                    yerr=probe_ds.ion_d_a, color=colours[i], mfc='none',
                    label=f'{probe_names[probe]}', marker=markers[i], ls='none')
#     ax.errorbar(probe_ds.theta, probe_ds.mf_iv_a, 
#                 yerr=probe_ds.mf_iv_d_a, color=colours[i],
#                 label=f'{probes[probe]}', marker=markers[i], ls='none')
    
    calced_a = lpu.calc_sheath_expansion_param(
        T_e, n_0, L * lambda_D, g * lambda_D, np.radians(dummy_theta), 
        c_1=0.9, c_2=0.6,
    )
    ax.errorbar(dummy_theta, calced_a, label=r'Predicted - Murphy-Sugrue', fmt='-', 
                color=cb_palette[3], linewidth=1.0)

    calced_a = lpu.calc_sheath_expansion_param(
        T_e, n_0, L * lambda_D, 0, np.radians(dummy_theta), 
        c_1=0.5, c_2=0.6,
    )
    ax.errorbar(dummy_theta, calced_a, label=r'Predicted - Bergmann', fmt='--', 
                color=cb_palette[2], linewidth=1.0)
    ax.set_title(title_str.format(title_vals[j]))
    ax.legend()

axes[0].set_ylim(3e-3, 1.5e-1)
axes[0].set_xlim(-0.5, 30.5)
axes[0].set_ylabel(r'$a$ [unitless]')
axes[0].set_xlabel(r'$\theta$ [$^{\circ}$]')
axes[1].set_xlabel(r'$\theta$ [$^{\circ}$]')

axes[0].legend().remove()


fig.tight_layout()

In [21]:
(fflwp_ds.sel(probe='flat_flush')['ion_a'] / hg_fflwp_ds.sel(probe='flat_flush')['ion_a']).mean()


<xarray.DataArray 'ion_a' ()>
array(1.05289359)
Coordinates:
    probe            <U10 'flat_flush'
    gap_desc         <U1 ''
    gap              float64 0.001
    recession        float64 0.0
    theta_p          float64 0.0
    theta_p_rads     float64 0.0
    recession_descr  object 'flush'
    theta_p_descr    object 'flat'
    recessed_fl      bool False
    angled_fl        bool False

In [55]:
# sheath exp for whole grid

sel_args = {'probe':'flat_flush'}


fig, ax = plt.subplots()

markers = ['o', '^', 's']
labels = [
    r'$V_w = 0$',
    r'$V_w = 0$',
#     r'$V_w = 2$',
    r'$V_w = 4 \sim V_{fl}$',
]
# datasets = [fwp0_ds, fwp2_ds, fflwp_ds]
# datasets = [fwp0_shexp_ds, fflwp_shexp_ds]
# datasets = [fwp0_ds, fwp0_shexp_ds]
datasets = [fflwp_ds, fflwp_shexp_ds]

for i, plot_ds in enumerate(datasets):

    plot_ds = plot_ds.sel(**sel_args)

    ax.errorbar('theta', 'ion_a', yerr='ion_d_a', data=plot_ds, marker=markers[i], 
                mfc='none', linestyle='none', label=labels[i], color=contrast_palette[i+1])



calced_a = lpu.calc_sheath_expansion_param(
    T_e, n_0, L * lambda_D, g * lambda_D, np.radians(dummy_theta), 
#     c_1=0.5, c_2=0.6,
    c_1=0.9, c_2=0.6,
)
ax.errorbar(dummy_theta, calced_a, label=r'Predicted - Murphy-Sugure', fmt='-', 
            color=paledark_palette[1][1], linewidth=1.0, alpha=0.6)

calced_a = lpu.calc_sheath_expansion_param(
    T_e, n_0, L * lambda_D, 0, np.radians(dummy_theta), 
    c_1=0.5, c_2=0.6,
#     c_1=0.9, c_2=0.6,
)
ax.errorbar(dummy_theta, calced_a, label=r'Predicted - Bergmann', fmt='--', 
            color=paledark_palette[1][2], linewidth=1.0, alpha=0.6)

ax.set_ylim(5e-3, 1.8e-1)
ax.set_xlim(-0.5, 30.5)
ax.set_ylabel(r'$a$ [unitless]')
ax.set_xlabel(r'$\theta$ [$^{\circ}$]')

ax.legend()
fig.tight_layout()

In [82]:
# sheath exp for half grid

sel_args = {'probe':'angled_recessed'}

fig, ax = plt.subplots()

markers = ['o', '^', 's']
labels = [
    r'$V_w = 0$',
#     r'$V_w = 2$',
    r'$V_w = 4 \sim V_{fl}$',
]

for i, plot_ds in enumerate([hg_ds, hg_fflwp_ds]):

    plot_ds = plot_ds.sel(**sel_args)

    ax.errorbar('theta', 'ion_a', yerr='ion_d_a', data=plot_ds, marker=markers[i], 
                mfc='none', linestyle='none', label=labels[i], 
                color=contrast_palette[i+2])



calced_a = lpu.calc_sheath_expansion_param(
    T_e, n_0, L * lambda_D, g * lambda_D, np.radians(dummy_theta), 
#     c_1=0.5, c_2=0.6,
    c_1=0.9, c_2=0.6,
)
ax.errorbar(dummy_theta, calced_a, label=r'Predicted - Murphy-Sugure', fmt='-', 
            color=paledark_palette[1][1], linewidth=1.0, alpha=0.6)

calced_a = lpu.calc_sheath_expansion_param(
    T_e, n_0, L * lambda_D, 0, np.radians(dummy_theta), 
    c_1=0.5, c_2=0.6,
#     c_1=0.9, c_2=0.6,
)
ax.errorbar(dummy_theta, calced_a, label=r'Predicted - Bergmann', fmt='--', 
            color=paledark_palette[1][2], linewidth=1.0, alpha=0.6)

ax.set_ylim(1e-3, 1.1e-1)
ax.set_xlim(-0.5, 30.5)
ax.set_ylabel(r'$a$ [unitless]')
ax.set_xlabel(r'$\theta$ [$^{\circ}$]')

ax.legend()
fig.tight_layout()

In [38]:

probes = ['flat_flush','angled_recessed']

fig, axes = plt.subplots(1, len(probes), figsize=[12,6])

markers = ['o', '^', 's']
labels = [
    r'$V_w = 0$',
#     r'$V_w = 2$',
    r'$V_w = V_{fl}$',
]

# # wholegrid coeffs
# probe_p_coeffs = {
#     'flat_flush': {'c_1': 1.3, 'c_2': 0.66},
#     'angled_recessed': {'c_1':1.0 , 'c_2': 0.45},
# #     'angled_recessed': {'c_1':-1.8 , 'c_2': 1.6}, 
# }
# probe_rr_coeffs = {
#     'flat_flush': {'c_1': 1.6, 'c_2': 0.56},
#     'angled_recessed': {'c_1':-0.62 , 'c_2': 1.8},
# #     'angled_recessed': {'c_1':-4.1 , 'c_2': 2.4},
# }


# halfgrid coeffs
probe_p_coeffs = {
    'flat_flush': {'c_1': 1.4, 'c_2': 0.47},
    'angled_recessed': {'c_1':1.6 , 'c_2': -0.21},
#     'angled_recessed': {'c_1':-1.8 , 'c_2': 1.6}, 
}
probe_rr_coeffs = {
    'flat_flush': {'c_1': 1.6, 'c_2': 0.41},
    'angled_recessed': {'c_1':1.6 , 'c_2': 0.3},
#     'angled_recessed': {'c_1':-4.1 , 'c_2': 2.4},
}

# datasets = [fwp0_ds, fwp2_ds, fflwp_ds]
datasets = [hg_ds, hg_fflwp_ds]

for j, probe in enumerate(probes):
    ax = axes[j]
    for i, plot_ds in enumerate(datasets):
        
        if probe in plot_ds.probe:
            plot_ds = plot_ds.sel(probe=probe)
        else:
            continue

        ax.errorbar(plot_ds['theta'], plot_ds['str_iv_a'], 
                    yerr=plot_ds['ion_d_a'], marker=markers[i], 
                    mfc='none', linestyle='none', label=labels[i], 
                    color=contrast_palette[i+2])



    calced_a = lpu.calc_new_sheath_expansion_param(
        T_e, n_0, L * lambda_D, g * lambda_D, np.radians(dummy_theta), 
        plot_ds.recession.values, plot_ds.theta_p_rads.values,
        **probe_p_coeffs[probe]
    )
    ax.errorbar(dummy_theta, calced_a, label=r'Parallelogrammatic', fmt='-', 
                color=paledark_palette[1][1], linewidth=1.0, alpha=0.6)

    calced_a = lpu.calc_2d_box_sheath_expansion_param(
        T_e, n_0, L * lambda_D, 0, np.radians(dummy_theta), 
        plot_ds.recession.values, plot_ds.theta_p_rads.values,
        **probe_rr_coeffs[probe]
    )
    ax.errorbar(dummy_theta, calced_a, label=r'Rotated Rectangular', fmt='--', 
                color=paledark_palette[1][2], linewidth=1.0, alpha=0.6)

    ax.set_ylim(1e-3, 0.7e-1)
    ax.set_xlim(-0.5, 40.5)
    ax.set_ylabel(r'$a$ [unitless]')
    ax.set_xlabel(r'$\theta_{tot}$ [$^{\circ}$]')

    ax.set_title(probe_names[probe])
    ax.legend()
fig.tight_layout()

# Decomposition to find coefficients

In [55]:
importlib.reload(dc)
importlib.reload(lpu)

<module 'flopter.core.lputils' from '/home/jleland/coding/projects/flopter/flopter/core/lputils.py'>

In [56]:
def cot(theta, theta_p=0.0):
    return 1 / np.tan(np.radians(theta) + theta_p)

def tan_p(theta, theta_p=0.0):
    return np.tan(np.radians(theta)) + (2 * np.tan(theta_p))

def reverse_cot(x, theta_p=0.0):
    return np.degrees(np.arctan(1/np.array(x)) - theta_p)

def reverse_tan_p(x, theta_p=0.0):
    return np.degrees(np.arctan(x - (2 * np.tan(theta_p))))

mode_funcs = [
    None, 
    (reverse_cot, cot),
    (reverse_tan_p, tan_p), 
    (reverse_cot, cot),
    (reverse_cot, cot),
]

def get_conv_func(theta_p, func):
    return lambda x : func(x, theta_p)

def get_mode_funcs(theta_p, mode):
    fwd_func, inv_func = mode_funcs[mode]
    return get_conv_func(theta_p, fwd_func), get_conv_func(theta_p, inv_func)

f = get_conv_func(0.0, tan_p)


In [57]:
f(2)

0.03492076949174773

In [58]:
DEFAULT_MODES = (1, 2, 3)
DEFAULT_COLOURS = contrast_palette[1:4]

kwargs_for_plot={
    'ls':'none', 
    'mfc':'none', 
    'marker': '.',
#     'label': None,
}
kwargs_for_fitplot={
    'ls':'none', 
    'mfc':'none', 
}

def plot_multi_decomp(plot_ds, probes=(), slices=(slice(None, None), ), sheath_label='ion_a', 
                      kwargs_for_plot=kwargs_for_plot, kwargs_for_fitplot=kwargs_for_fitplot, 
                      colours=DEFAULT_COLOURS, modes=DEFAULT_MODES, theta_scale_fl=False, axes=None):
    probes_len = len(probes)
    modes_len = len(modes)
    if axes is None:
        fig, axes = plt.subplots(modes_len, probes_len, figsize=[6*probes_len, 5*modes_len])
    else:
        fig = plt.gca().figure
    
    for xax_i, probe in enumerate(probes):
        whole_ds = plot_ds.sel(probe=probe)
        plot_slice_dss = [plot_ds.sel(probe=probe, theta=slc) for slc in slices]
        
        if probes_len == 1:
            ax = axes
        else:
            ax = axes[:, xax_i]
        
        for yax_j, mode in enumerate(modes):
            dc.plot_decomp(whole_ds, sheath_label=sheath_label, plot_label=None, mode=mode, ax=ax[yax_j],
                           colour=colours[0], kwargs_for_plot=kwargs_for_plot, fit_fl=False)
            for k, psds in enumerate(plot_slice_dss):
                dc.plot_decomp(psds, sheath_label=sheath_label, plot_label=None, mode=mode, ax=ax[yax_j],
                               colour=colours[k+1], kwargs_for_plot=kwargs_for_plot)

        ax[0].set_title(probe_names[probe])
        
    fig.tight_layout()
    return axes

In [59]:
# Prototype Decomposition Plot for getting Theta axis to work

probe = 'flat_flush'
plot_ds = hg_ds

plot_hg_whole_ds = plot_ds.sel(probe=probe)
plot_hg_ds = plot_ds.sel(probe=probe, theta=slice(0, 30))
plot_hg_ds = plot_ds.sel(probe=probe, theta=slice(10, 30))


fig, ax = plt.subplots(3, figsize=[6,15])
dc.plot_decomp(plot_hg_ds, sheath_label='ion_a', mode=1, ax=ax[0], 
               colour=cb_palette[0], kwargs_for_plot=kwargs_for_plot)
dc.plot_decomp(plot_hg_whole_ds, sheath_label='ion_a', mode=1, ax=ax[0], 
               colour=cb_palette[2],
               kwargs_for_plot=kwargs_for_plot, fit_fl=False)

dc.plot_decomp(plot_hg_ds, sheath_label='ion_a', mode=2, ax=ax[1], colour=cb_palette[0], 
               kwargs_for_plot=kwargs_for_plot)
dc.plot_decomp(plot_hg_whole_ds, sheath_label='ion_a', mode=2, ax=ax[1], 
               colour=cb_palette[2], kwargs_for_plot=kwargs_for_plot, fit_fl=False)

dc.plot_decomp(plot_hg_ds, sheath_label='ion_a', mode=3, ax=ax[2], colour=cb_palette[0], 
               kwargs_for_plot=kwargs_for_plot)
dc.plot_decomp(plot_hg_whole_ds, sheath_label='ion_a', mode=3, ax=ax[2], 
               colour=cb_palette[2], kwargs_for_plot=kwargs_for_plot, fit_fl=False)

ax[0].set_xlim(0.5, 30.5)
ax_theta = ax[0].twiny()
x_lims = ax[0].get_xlim()
print(x_lims)

fwd_f, inv_f = get_mode_funcs(0.0, 1)
theta_lims = fwd_f(x12)

print(theta_lims)

ax_theta.set_xscale('function', functions=(fwd_f, inv_f))
ax_theta.set_xticks([1, 2, 5, 10, 30])
ax_theta.set_xlim(*x_lims)


ax[0].set_title(probe)
fig.tight_layout()

300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0
(0.5, 30.5)


NameError: name 'x12' is not defined

In [60]:
plot_multi_decomp(hg_fflwp_ds, probes=('flat_flush', 'angled_recessed'), slices=(slice(20,30), slice(2,10)))

No handles with labels found to put in legend.
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/scipy/optimize/minpack.py:795: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.


300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0
300 60 17.999999999999996 10.0
300 60 17.999999999999996 10.0
300 60 17.999999999999996 10.0
300 60 17.999999999999996 10.0
300 60 17.999999999999996 10.0
300 60 17.999999999999996 10.0


No handles with labels found to put in legend.


300 60 17.999999999999996 10.0
300 60 17.999999999999996 10.0
300 60 17.999999999999996 10.0


array([[<matplotlib.axes._subplots.AxesSubplot object at 0x7f6159f15e10>,
      dtype=object)

In [61]:
# Decomposition

probe = 'flat_flush'
plot_ds = hg_ds

plot_multi_decomp(hg_ds, probes=[probe], slices=[slice(0, 6), slice(15, 30)])



No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.


300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0


array([<matplotlib.axes._subplots.AxesSubplot object at 0x7f61603d1940>,
      dtype=object)

In [48]:
probe = 'flat_flush'
fig, axes = plt.subplots(3, 3, sharey='row', figsize=[18,15])

for i, plot_ds in enumerate([fwp0_ds, fwp2_ds, fflwp_ds]):
    plot_multi_decomp(plot_ds, probes=[probe], slices=[slice(0, 6), slice(15, 30)], axes=axes[:,i])


No handles with labels found to put in legend.
No handles with labels found to put in legend.


300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0


No handles with labels found to put in legend.


300 60 0.0 0.0
300 60 0.0 0.0


No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.


300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0


No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.


300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0


In [62]:
importlib.reload(lpu)
importlib.reload(dc)

dc.DEFAULT_DEBYE


1.6666666666666667e-05

In [65]:
# plot_ds = fflwp_ds
plot_ds = fflwp_shexp_ds
probe_slices = {'flat_flush':[slice(4,30)], 'angled_recessed':[slice(4,30)]}
colours = [[cb_palette[-1], cb_palette[0]], [cb_palette[-1], cb_palette[4]], [cb_palette[-1], cb_palette[3]]]

fig, axes = plt.subplots(2, 2, sharey='row', figsize=[8,8])

for i, (probe, slc) in enumerate(probe_slices.items()):
    plot_multi_decomp(plot_ds, probes=[probe], slices=slc, axes=axes[:,i], 
                      colours=colours[i], modes=[3,4])
    if i != 0:
        for ax in axes[:,i]:
            ax.set_ylabel('')
fig.tight_layout()

No handles with labels found to put in legend.
No handles with labels found to put in legend.


300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0


No handles with labels found to put in legend.
No handles with labels found to put in legend.


300 60 17.999999999999996 10.0
300 60 17.999999999999996 10.0
300 60 17.999999999999996 10.0
300 60 17.999999999999996 10.0


In [132]:
# plot_ds = fflwp_ds
plot_ds = hg_fflwp_ds
probe_slices = {'flat_flush':[slice(5,30)], 'angled_recessed':[slice(5,30)]}
colours = [[cb_palette[-1], cb_palette[0]], [cb_palette[-1], cb_palette[4], cb_palette[5]]]

fig, axes = plt.subplots(2, 2, sharey='row', figsize=[12,12])

for i, (probe, slc) in enumerate(probe_slices.items()):
    plot_multi_decomp(plot_ds, probes=[probe], slices=slc, axes=axes[:,i], 
                      colours=colours[i], modes=[3,4])

(2,)
300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0
300 60 0.0 0.0


(2,)
300 60 17.999999999999996 10.0
300 60 17.999999999999996 10.0
300 60 17.999999999999996 10.0
300 60 17.999999999999996 10.0


In [ ]:
plot_ds = hg_fflwp_ds
probe_slices = {'flat_flush':[slice(8,30)], 'angled_recessed':[slice(11,30), slice(2,10)]}
colours = [[cb_palette[-1], cb_palette[0]], [cb_palette[-1], cb_palette[4], cb_palette[5]]]

fig, axes = plt.subplots(2, sharey='row', figsize=[8,12])

for i, (probe, slc) in enumerate(probe_slices.items()):
    plot_multi_decomp(plot_ds, probes=[probe], slices=slc, axes=axes, 
                      colours=colours[i], modes=[2,3])

## Looking at wall potential and critical angles

In [66]:
def calc_critical_angle(u_wall):
    return np.degrees(np.arcsin(np.exp(u_wall)))

def calc_u_wall(theta_c):
    return np.log(np.sin(np.radians(theta_c)))

def calc_floating_pot(tau, mu):
    return 0.5 * np.log((2*np.pi/mu)*(1+tau))

print(calc_critical_angle(-2))

7.778008133987804


In [67]:
print(calc_u_wall(7.2))    # peak for V_w = -4
print(calc_u_wall(9.6))    # peak for V_w = -2
print(calc_u_wall(11))     # peak for V_w = 0

-2.0767792202901565
-1.7911471764209466
-1.6564823754998934


In [68]:
ideal_floating_pot = calc_floating_pot(1, 1836)

print(ideal_floating_pot)
print(np.degrees(np.arcsin(np.sqrt(np.exp(2*ideal_floating_pot)))))

-2.4921601621055727
4.74556460611768


In [69]:
dummy_u = np.linspace(-100,0,1000)
theta_c = calc_critical_angle(dummy_u)

fig, ax = plt.subplots()
ax.plot(dummy_u, theta_c)
ax.set_xlim(-5,-0.5)


(-5, -0.5)

In [70]:
sel_args = {'probe':'flat_flush'}


fig, ax = plt.subplots()

markers = ['o', '^', 's']
labels = [
    r'$V_w = 0$',
    r'$V_w = 2$',
    r'$V_w = 4 \sim V_{fl}$',
]

for i, plot_ds in enumerate([fwp0_ds, fwp2_ds, fflwp_ds]):

    plot_ds = plot_ds.sel(**sel_args)

    ax.plot(plot_ds['theta'], plot_ds['str_iv_V_f'] - ideal_floating_pot, 
            marker=markers[i], mfc='none', linestyle='none', label=labels[i], 
            color=contrast_palette[i+1])


ax.set_ylabel(r'$a$ [unitless]')
ax.set_xlabel(r'$\theta$ [$^{\circ}$]')

ax.legend()
fig.tight_layout()


In [71]:
fig, ax = plt.subplots(2, 2, sharey='row', sharex=True, figsize=[10,6])

# runs = ['fwp_0', 'fwp_2', 'fflwp']
runs = ['hg_fflwp', 'fflwp']
labels = [
#     r'$V_w = 0$ (HR)',
#     r'$V_w = 2$',
#     r'$V_w = 4 \sim V_{fl}$ (HR)',
#     r'$V_w = 0$ (LR)',
#     r'$V_w = 4 \sim V_{fl}$ (LR)',
    r'$dz = dy = 0.5$',
    r'$dz = dy = 1.0$',
]
denormers = [
    denormaliser_hg, denormaliser_wg, denormaliser_wg
]

plot_ds = combined_ds.sel(run=runs, probe='flat_flush').set_coords('voltage_corr')
plot_ds = plot_ds.where(plot_ds.current < 0, drop=True)
thetas = [8.0, 30.0]

for i, run in enumerate(plot_ds.run.values):
    for j, theta in enumerate(thetas): 
        run_ds = plot_ds.sel(run=run, theta=theta)
        ax[1, j].plot(run_ds.voltage, denormers[i](run_ds.current, 'current'), 
                   color=contrast_palette[i+1], label=labels[i])
        ax[0, j].plot(run_ds.voltage, run_ds.current, color=contrast_palette[i+1], label=labels[i])
        ax[0, j].set_title(r'$\theta=${}'.format(theta))
    

        
# ax[0].set_ylim(5e-3, 5e-2)
ax[0, 0].set_xlim(-15, -0)

ax[0, 0].set_ylabel(r'$\hat{I}_{tot}$')
ax[1, 0].set_ylabel(r'Denormalised ${I}_{tot} [A]$')
ax[1, 0].set_xlabel(r'$\hat{V} \;\; [e/k_B T_e]$')
ax[1, 1].set_xlabel(r'$\hat{V} \;\; [e/k_B T_e]$')

for axis in ax[:,:]:
    axis[0].axhline(y=0, **c.AX_LINE_DEFAULTS)
    axis[1].axhline(y=0, **c.AX_LINE_DEFAULTS)

ax[0, 0].legend()
fig.tight_layout()

In [77]:
fig, ax = plt.subplots(1, 2, sharey='row', sharex=True, figsize=[10,6])

# runs = ['fwp_0', 'fwp_2', 'fflwp']
runs = ['fwp_0', 'fflwp']
labels = [
#     r'$V_w = 0$ (HR)',
#     r'$V_w = 2$',
#     r'$V_w = 4 \sim V_{fl}$ (HR)',
    r'$V_w = 0$',
    r'$V_w = V_{fl}$',
#     r'$dz = dy = 0.5$',
#     r'$dz = dy = 1.0$',
]
denormers = [
#     denormaliser_hg, denormaliser_wg, denormaliser_wg
    denormaliser_wg, denormaliser_wg
]
probe = 'angled_recessed'
plot_ds = combined_shexp_ds.sel(run=runs, probe=probe).set_coords('voltage_corr')
plot_ds = plot_ds.where(plot_ds.current < 0, drop=True)
thetas = [6.0, 16.0]

for i, run in enumerate(plot_ds.run.values):
    for j, theta in enumerate(thetas): 
        run_ds = plot_ds.sel(run=run, theta=theta)
        ax[j].plot(run_ds.voltage_corr, denormers[i](run_ds.current, 'current'), 
                   color=contrast_palette[i+2], label=labels[i])
#         ax[0, j].plot(run_ds.voltage, run_ds.current, color=contrast_palette[i+1], label=labels[i])
        ax[j].set_title(r'$\theta=${}'.format(theta))
    

        
# ax[0].set_ylim(5e-3, 5e-2)
ax[0].set_xlim(-10, -0)

ax[0].set_ylabel(r'Denormalised ${I}_{tot} [A]$')
ax[0].set_xlabel(r'$\hat{V} \;\; [e/k_B T_e]$')
ax[1].set_xlabel(r'$\hat{V} \;\; [e/k_B T_e]$')

for axis in ax:
    axis.axhline(y=0, ls='-', lw='0.8', color='k')

ax[0].legend()
fig.tight_layout()

In [78]:
print(denormaliser_hg(1, 'current'))
print(denormaliser_wg(1, 'current'))

0.09990109679420577
0.19980219358841153


In [79]:
print(0.5 * 0.5 * lambda_D**2 * lpu.ion_gyrofrequency(0.8) * c.ELEM_CHARGE * n_0 / (denormaliser_hg.dt * 50))
print(lambda_D**2 * lpu.ion_gyrofrequency(0.8) * c.ELEM_CHARGE * n_0 / (denormaliser_wg.dt * 50))

0.09989279002559179
0.19978558005118358


In [80]:
dummy_theta_coarse = np.linspace(1.5,15,100)


fig, ax = plt.subplots()

for voltage in np.linspace(10, 10, 7):
    sheath_height = 0.5 * np.float_power(np.abs(voltage), 0.75) / np.sqrt(np.sin(np.radians(dummy_theta_coarse)))
    ax.plot(dummy_theta_coarse, sheath_height / np.tan(np.radians(dummy_theta_coarse)), label=f'{voltage}')
ax.axhline(y=136, **c.AX_LINE_DEFAULTS)
ax.set_xlabel('Required window width')
ax.legend()

In [146]:
c_s = lpu.sound_speed(T_e, gamma_i=2)
print(c_s)
I_0_bits =  (c.ELEM_CHARGE * n_0 * c_s)

L_mps = np.sqrt(6) * (c_s / lpu.ion_gyrofrequency(0.8))
larmor_radius = lpu.ion_larmor_radius(T_e, 0.8)

37905.47429048637


In [113]:
# plot_ds = combined_ds.sel(run=['fwp_0', 'fwp_2', 'fflwp'], probe='flat_flush')
probes = ['flat_flush', 'angled_recessed']

# datasets = [fwp0_ds, fwp2_ds, fflwp_ds]
datasets = [hg_ds, hg_fflwp_ds]

fig, ax = plt.subplots(1, 2, sharex=True, sharey=True, figsize=[10,6])
for i, probe in enumerate(probes):
    for j, plot_ds in enumerate(datasets):
        if probe not in plot_ds.probe.values:
            continue
        plot_ds = plot_ds.sel(probe=probe)
        I_0 = denormaliser_wg(plot_ds.ion_I_sat.values, 'current') / I_0_bits
        
        lhs = (I_0 - (L + g)*lambda_D*np.sin(plot_ds.theta_rads) )/ np.cos(plot_ds.theta_rads)
        
        
        ax[i].plot(plot_ds.theta, lhs, 'x', color=contrast_palette[j+1], label=labels[j])
        ax[i].set_title(probe_names[probe])
        ax[i].legend()
#         plot_ds['ion_I_sat'].plot.line(x='theta', color=contrast_palette[j+1], ax=ax[i])mpl

In [84]:
# Delta_0 graph for either hg or wg, probes split across axes

fig, ax = plt.subplots(1, 2, sharex=True, sharey=True, figsize=[10,6])

denormaliser = denormaliser_hg
base_ds = hg_fflwp_ds
plot_datasets = [hg_ds]

# denormaliser = denormaliser_wg
# base_ds = fflwp_ds
# plot_datasets = [fwp0_ds, fwp2_ds]

delta_0s = []

for i, probe in enumerate(probes):
    base_probe_ds = base_ds.sel(probe=probe)
#     ax[i].plot(dummy_theta, get_sheath_size(0.4, dummy_theta, 2))
#     ax[i].plot(dummy_theta, get_sheath_size(2.4, dummy_theta, 2))
    for j, plot_ds in enumerate(plot_datasets):
        if probe not in plot_ds.probe.values:
            continue
        plot_ds = plot_ds.sel(probe=probe)
        
        thetas = list(set(plot_ds.theta.values) & set(base_probe_ds.theta.values))
        base_plot_ds = base_probe_ds.sel(theta=thetas)
        plot_ds = plot_ds.sel(theta=thetas)
        
        I_diff = denormaliser(plot_ds.ion_I_sat.values - base_plot_ds.ion_I_sat.values, 'current') / I_0_bits
        delta_0 = I_diff / np.cos(plot_ds.theta_rads) /lambda_D
        delta_0s.append(delta_0)
        
        ax[i].plot(plot_ds.theta, delta_0, 
                   color=contrast_palette[j+2], label=labels[j], ls='none', marker=markers[j])
#         ax[i].plot(plot_ds.theta, (I_diff / np.cos(plot_ds.theta_rads) /lambda_D) - get_sheath_size(2.4, plot_ds.theta, 2), 
#                    color=cb_palette[j], ls='none', marker=markers[j])
    
    ax[i].axhline(y=larmor_radius/lambda_D, **c.AX_LINE_DEFAULTS)
    ax[i].set_title(probe_names[probe])
    ax[i].legend()
#     ax[i].set_xlim(1,30.5)
#     ax[i].set_ylim(-0.5,18.5)
    
ax[0].set_xlabel(r'$\theta \;[\degree]$')
ax[1].set_xlabel(r'$\theta \;[\degree]$')
ax[0].set_ylabel(r'$\Delta_0 - \Delta_{w}(V=0) \; [\lambda_D]$')
fig.tight_layout()

In [147]:
def get_density(i_sat, L_0, denorm, c_s=c_s, corr=0.0):
    return denorm(i_sat, 'current') / (c.ELEM_CHARGE *  c_s * lambda_D * (L_0 + corr))

def get_mps_density(angle, n_ds):
    return n_ds / np.sin(angle)

In [89]:
def get_sheath_size(c_1, theta, V_w, phi=0.0):
    return (c_1/np.sqrt(np.sin(np.radians(theta)))) * np.float_power(np.abs(phi - V_w), 0.75)

def get_L_0_p(L, g, theta, theta_p, d_perp):
    return (L+g) * np.sin(theta) + (L*np.tan(theta_p) - d_perp) * np.cos(theta)

def get_L_0_rr(L, g, theta, theta_p, d_perp, delta_0=0.0):
    return ((L+g) * np.sin(theta) 
            + (L*np.tan(theta_p) - d_perp) * np.cos(theta)
            + (delta_0 * np.cos(theta) * (np.cos(theta_p)*(1 - np.tan(theta)*np.tan(theta_p)) - 1))
           )

In [87]:
# From piecewise notebook
thetas_pw = np.array([ 3.,  5.,  8., 10., 12., 15., 20.])
delta_0_pw = np.array([3.48223737, 8.12342498, 9.18608777, 8.7844804, 9.66204131, 10.10223821, 8.68876167])
d_delta_0_pw = np.array([1.13982315, 0.60212275, 0.7579872 , 0.59098696, 0.47533765, 0.63424856, 0.59817238])

pw_slc = slice(1,None)
thetas_pw[pw_slc]

array([ 5.,  8., 10., 12., 15., 20.])

In [355]:
# Delta_0 and Density graph for either hg or wg (uncomment relevant preamble), probes split across axes

fig, ax = plt.subplots(2, 2, sharex=True, sharey='row', figsize=[10,10])

probes = ['flat_flush', 'angled_recessed']

# denormaliser = denormaliser_hg
# base_ds = hg_fflwp_ds
# plot_datasets = [hg_ds]

denormaliser = denormaliser_wg
base_ds = fflwp_shexp_ds
plot_datasets = [fwp0_shexp_ds]

labels = [
    r'$V_w = 0$',
#     r'$V_w = 2$',
    r'$V_w = V_{fl}$',
]

corr_markers = ['x', '+']
delta_0s = []
d_delta_0s = []
plotted_thetas = []

# ax[0, 0].errorbar(thetas_pw[pw_slc], delta_0_pw[pw_slc], yerr=d_delta_0_pw[pw_slc], 
#                   color=cb_palette[2], fmt='s', mfc='none', label=labels[0]+'(Piecewise)')
# ax[0, 1].errorbar(thetas_pw[pw_slc], delta_0_pw[pw_slc], yerr=d_delta_0_pw[pw_slc], 
#                   color=cb_palette[2], fmt='s', mfc='none', label=labels[0]+'(Piecewise)')


for i, probe in enumerate(probes):
    base_probe_ds = base_ds.sel(probe=probe, theta=slice(5,None))
    L_0 = get_L_0_rr(L, g, base_probe_ds.theta_rads, base_probe_ds.theta_p_rads, 
                     base_probe_ds.recession/lambda_D, 0.0) 
    uncorr_dens = (denormaliser(base_probe_ds.ion_I_sat.values, 'current') 
                   / (c.ELEM_CHARGE *  c_s * lambda_D * L_0))
    d_uncorr_dens = denormaliser(plot_ds['ion_d_I_sat'].values, 'current') / (c.ELEM_CHARGE *  c_s * lambda_D * L_0)
    ax[1, i].errorbar(base_probe_ds.theta, uncorr_dens, color=contrast_palette[3], mfc='none', yerr=d_uncorr_dens,
                     label=labels[1], ls='none', marker=markers[2])

#     ax[i].plot(dummy_theta, get_sheath_size(0.4, dummy_theta, 2))
#     ax[i].plot(dummy_theta, get_sheath_size(2.4, dummy_theta, 2))
    for j, plot_ds in enumerate(plot_datasets):
        if probe not in plot_ds.probe.values:
            continue
        plot_ds = plot_ds.sel(probe=probe, theta=slice(5,None))
        
        thetas = list(set(plot_ds.theta.values) & set(base_probe_ds.theta.values))
        base_plot_ds = base_probe_ds.sel(theta=thetas)
        plot_ds = plot_ds.sel(theta=thetas)
        
        I_diff = denormaliser(plot_ds['ion_I_sat'].values - base_plot_ds['ion_I_sat'].values, 'current') / I_0_bits
        d_I_diff = np.sqrt(denormaliser(base_plot_ds['ion_d_I_sat'].values, 'current')**2 
                           + denormaliser(base_plot_ds['ion_d_I_sat'].values, 'current')**2) / I_0_bits
        delta_0 = I_diff / np.cos(plot_ds.theta_rads) / lambda_D
        d_delta_0 = d_I_diff / np.cos(plot_ds.theta_rads) / lambda_D
        
        delta_0s.append(delta_0)
        d_delta_0s.append(d_delta_0)
        plotted_thetas.append(thetas)
        
        ax[0, i].errorbar(plot_ds.theta, delta_0, yerr=d_delta_0, mfc='none',
                          color=contrast_palette[j+2], label=labels[j], ls='none', marker=markers[j])
        
#         ax[0, i].errorbar(plot_ds.theta, L_mps * np.cos(plot_ds.theta_rads) / lambda_D, 
#                           color=cb_palette[6], fmt='s', mfc='none', label=labels[0]+'(Piecewise)')
        
        L_0 = get_L_0_rr(L, g, plot_ds.theta_rads, plot_ds.theta_p_rads, plot_ds.recession/lambda_D, ) 
#         print(L_0)
        corr_dens = denormaliser(plot_ds['ion_I_sat'].values, 'current') / (c.ELEM_CHARGE *  c_s * lambda_D * (L_0 + ((delta_0) * np.cos(plot_ds.theta_rads))))
        uncorr_dens = denormaliser(plot_ds['ion_I_sat'].values, 'current') / (c.ELEM_CHARGE *  c_s * lambda_D * L_0)
        
        d_uncorr_dens_a = denormaliser(plot_ds['ion_d_I_sat'].values, 'current') / (c.ELEM_CHARGE *  c_s * lambda_D)
        d_uncorr_dens = d_uncorr_dens_a / L_0
        d_corr_dens = corr_dens * np.sqrt((plot_ds['ion_d_I_sat'].values/plot_ds['ion_I_sat'].values)**2 + 
                                         ((d_delta_0 * np.cos(plot_ds.theta_rads))/(L_0 + (delta_0 * np.cos(plot_ds.theta_rads))))**2)
                                          
#         L_0_pw = get_L_0_rr(L, g, np.radians(thetas_pw), plot_ds.theta_p_rads.values, plot_ds.recession.values/lambda_D, 0.0) 

#         corr_pw_dens = denormaliser(plot_ds.sel(theta=thetas_pw).ion_I_sat.values, 'current') / (c.ELEM_CHARGE *  c_s * lambda_D * (L_0_pw + ((delta_0_pw) * np.cos(np.radians(thetas_pw)))))
        
        ax[1, i].errorbar(plot_ds.theta, uncorr_dens, yerr=d_uncorr_dens, mfc='none',
                          color=contrast_palette[j+2], label=labels[j], ls='none', marker=markers[j])
        ax[1, i].errorbar(plot_ds.theta, corr_dens, yerr=d_corr_dens, mew=2,
                          color=contrast_palette[j+2], label=labels[j] + '(Corrected)', ls='none', marker=corr_markers[j])
#         ax[1, i].plot(plot_ds.theta, get_mps_density(plot_ds.theta_rads, corr_dens), mew=2,
#                       color=contrast_palette[j+1], label=labels[j] + '(Corrected)', ls='none', marker=corr_markers[j])
#         ax[1, i].plot(thetas_pw, corr_pw_dens, mfc='none',
#                       color=cb_palette[j+1], label=labels[j] + '(Corrected)', ls='none', marker=markers[j])
    
    ax[0, i].axhline(y=larmor_radius/lambda_D, **c.AX_LINE_DEFAULTS, label=r'$\rho_{L,i}$')
    ax[1, i].axhline(y=n_0, **c.AX_LINE_DEFAULTS, label=r'$n_0$')
    ax[0, i].set_title(probe_names[probe])
    ax[0, i].legend()
    ax[1, i].legend()



ax[1, 0].set_xlabel(r'$\theta \;[\degree]$')
ax[1, 1].set_xlabel(r'$\theta \;[\degree]$')
ax[0, 0].set_ylabel(r'$\Delta_0 - \Delta_{w}(V=0) \; [\lambda_D]$')
ax[1, 0].set_ylabel(r'$n_e \; [m^{-3}]$')
fig.tight_layout()

In [175]:
fig, ax = plt.subplots()

ax.plot(dummy_theta, get_L_0_p(L, g, np.radians(dummy_theta), np.radians(10.0), 20.0))
ax.plot(dummy_theta, get_L_0_rr(L, g, np.radians(dummy_theta), np.radians(10.0), 20.0, delta_0=10.0))

In [361]:
fig, ax = plt.subplots(1, 2, figsize=[10,5], sharex=True, sharey=True)

colours = [cb_palette[0], cb_palette[4]]
sl_fitter = fts.StraightLineFitter()
for i, probe in enumerate(probes):
    floating_pot = fflwp_shexp_ds.sel(theta=plotted_thetas[i], probe=probe)['voltage_wall'].mean('voltage')
    d_floating_pot = fflwp_shexp_ds.sel(theta=plotted_thetas[i], probe=probe)['voltage_wall'].std('voltage')
    
    x_reduction = np.float_power(np.abs(floating_pot), 0.75) / np.sqrt(np.sin(np.radians(plotted_thetas[i])))
    
    ax[i].errorbar(x_reduction, delta_0s[i], yerr=d_delta_0s[i], color=colours[i], 
                   marker=markers[i], ls='none', label=probe_names[probe], mfc='none')
    
    f_data = sl_fitter.fit(x_reduction, delta_0s[i], sigma=d_delta_0s[i])
#     f_data.plot(ax=ax)
#     delta_0_guess = get_sheath_size(2.5, plotted_thetas[i], floating_pot)
    f_label = r'$c$={:.2g}$\pm${:.1g}'.format(f_data.get_param('m'), f_data.get_param_err('m'))
    ax[i].plot(*f_data.get_fit_plottables(), label=f_label, color=colours[i], 
               ls=linestyles[i])
    
    ax[i].legend()
    ax[i].set_xlabel(r'$|V_{fl}|^{3/4} / \sqrt{\sin{\theta}}$')
ax[0].set_ylabel(r'$\Delta_0 - \Delta_{w}(V=0) \; [\lambda_D]$')

fig.tight_layout()

In [43]:
# rho_L = larmor_radius / lambda_D
theta = plotted_thetas[0]
rho_L = delta_0s[0]
rho_L = get_sheath_size(2.0, theta, V_w=2)

fig, ax = plt.subplots()

L_0 = get_L_0_rr(L, g, np.radians(theta), np.radians(10.0), 3e-4/lambda_D, 0)
percentage = rho_L * np.cos(np.radians(theta)) / ((rho_L * np.cos(np.radians(theta))) + L_0 )
ax.plot(theta, percentage)

In [44]:
# Delta_0 graph for both hg and wg split across axes, probes grouped

fig, ax = plt.subplots(1, 2, sharex=True, sharey=True, figsize=[10,6])

floating_datasets = [fflwp_ds, hg_fflwp_ds]
sheathless_datasets = [fwp0_ds, hg_ds]
denormalisers = [denormaliser_wg, denormaliser_hg]
dataset_labels = ['Lower resolution', 'Higher resolution']

for i, fl_ds in enumerate(floating_datasets):
    
    for j, probe in enumerate(probes):
        base_ds = fl_ds.sel(probe=probe)
        plot_ds = sheathless_datasets[i]
    
        if probe not in plot_ds.probe.values:
            continue    
        plot_ds = plot_ds.sel(probe=probe)
        
        thetas = list(set(plot_ds.theta.values) & set(base_ds.theta.values))
        base_ds = base_ds.sel(theta=thetas)
        plot_ds = plot_ds.sel(theta=thetas)
        
        I_diff = denormalisers[i](plot_ds.ion_I_sat.values - base_ds.ion_I_sat.values, 'current') / I_0_bits
        
        ax[i].plot(plot_ds.theta, I_diff / np.cos(plot_ds.theta_rads) /lambda_D, 
                   color=contrast_palette[j+2], label=probe_names[probe], ls='none', marker=markers[j])
#         ax[i].plot(plot_ds.theta, (I_diff / np.cos(plot_ds.theta_rads) /lambda_D) - get_sheath_size(2.4, plot_ds.theta, 2), 
#                    color=cb_palette[j], ls='none', marker=markers[j])
    
    ax[i].axhline(y=larmor_radius/lambda_D, **c.AX_LINE_DEFAULTS, label=r'$\rho_{L,i}$')
    ax[i].set_title(dataset_labels[i])
    
    
ax[0].set_xlabel(r'$\theta$')
ax[1].set_xlabel(r'$\theta$')
ax[0].set_ylabel(r'$\Delta_0 - \Delta_{w}(V=0) \; [\lambda_D]$')
ax[1].legend()
#     ax[i].set_xlim(1,30.5)
#     ax[i].set_ylim(-0.5,18.5)

In [354]:
# Uncorrected Density graph for either hg or wg (uncomment relevant preamble), probes split across axes

fig, ax = plt.subplots(1, 2, sharex=True, sharey='row', figsize=[10,6])

c_s = lpu.sound_speed(T_e, gamma_i=2)
I_0_bits =  (c.ELEM_CHARGE * n_0 * c_s)

denormaliser = denormaliser_wg
# datasets = [fwp0_ds, fwp2_ds, fflwp_ds]
datasets = [fwp0_shexp_ds, fflwp_shexp_ds]

probes = ['flat_flush', 'angled_recessed']
probe_obj = [ff_probe, ar_probe]
# denormaliser = denormaliser_hg
# datasets = [hg_ds, fwp2_ds, hg_fflwp_ds]

labels = [
    r'$V_w = 0$',
#     r'$V_w = 2$',
    r'$V_w = V_{fl}$',
]


for i, probe in enumerate(probes):
    for j, plot_ds in enumerate(datasets):
        if probe not in plot_ds.probe.values:
            continue
        plot_ds = plot_ds.sel(probe=probe, theta=slice(5,None))
        
        L_0 = get_L_0_rr(L, g, plot_ds.theta_rads, plot_ds.theta_p_rads.values, plot_ds.recession/lambda_D, 0.0) 
#         L_0 = probe_obj[i].get_2d_collection_length(plot_ds.theta_rads.values) / lambda_D
        
        uncorr_dens = get_density(plot_ds.ion_I_sat.values, L_0, denormaliser, c_s=c_s)
        d_uncorr_dens_a = denormaliser(plot_ds['ion_d_I_sat'].values, 'current') / (c.ELEM_CHARGE *  c_s * lambda_D * L_0)
#         corr_dens = (denormaliser(plot_ds.ion_I_sat.values, 'current') / (c.ELEM_CHARGE *  c_s * lambda_D * (L_0 + ((delta_0) * np.cos(plot_ds.theta_rads)))))
        
#         test_L_0 = ((L+g)*np.sin(plot_ds.theta_rads)) + ((L*np.tan(plot_ds.theta_p_rads)) - (plot_ds.recession/lambda_D))*np.cos(plot_ds.theta_rads)
#         print(L_0.values - test_L_0.values)

#         test_dens = plot_ds.ion_I_sat / (c.ELEM_CHARGE *  c_s * lambda_D * test_L_0)
    
        ax[i].errorbar(plot_ds.theta, np.abs(uncorr_dens), color=contrast_palette[j+2], yerr=d_uncorr_dens,
                       label=labels[j], ls='none', marker=markers[j], mfc='none')
        
    
    ax[i].axhline(y=n_0, **c.AX_LINE_DEFAULTS, label=r'$n_0$')
    ax[i].set_title(probe_names[probe])
    ax[i].legend()
    ax[i].legend()
    
ax[0].set_xlabel(r'$\theta \;[\degree]$')
ax[1].set_xlabel(r'$\theta \;[\degree]$')
ax[0].set_ylabel(r'$n_e \; [\lambda_D]$')
fig.tight_layout()

In [302]:
# Uncorrected Density graph comparing hg and wg - flat_flush


fig, ax = plt.subplots(1, 2, sharex=True, figsize=[10,6])
# ax=[ax]

probes = ['flat_flush']
denormaliser = [denormaliser_hg, denormaliser_wg]
datasets = [hg_fflwp_ds, fflwp_ds]
labels = [
    r'$dz = dy = 0.5$',
    r'$dz = dy = 1.0$',
]

uncorr_thetas = []
uncorr_denss = []

for i, probe in enumerate(probes):
    for j, plot_ds in enumerate(datasets):
        if probe not in plot_ds.probe.values:
            continue
        plot_ds = plot_ds.sel(probe=probe, theta=slice(3,30))
        
        L_0 = get_L_0_rr(L, g, plot_ds.theta_rads, plot_ds.theta_p_rads.values, plot_ds.recession/lambda_D, 0.0) 
        
        uncorr_dens = get_density(plot_ds.ion_I_sat.values, L_0, denormaliser[j])
        uncorr_thetas.append(plot_ds.theta.values)
        uncorr_denss.append(uncorr_dens.values)
    
        ax[i].plot(plot_ds.theta, uncorr_dens, color=contrast_palette[j+2], 
                   label=labels[j], ls='none', marker=markers[j])
    
    ax[i].axhline(y=n_0, **c.AX_LINE_DEFAULTS, label=r'$n_0$')
    ax[i].set_title(probe_names[probe])
    ax[i].legend()
    ax[i].legend()
    
ax[1].plot(plot_ds.theta, uncorr_denss[0] / uncorr_denss[1], marker='x')
    
ax[0].set_xlabel(r'$\theta \;[\degree]$')
# ax[1].set_xlabel(r'$\theta \;[\degree]$')
ax[0].set_ylabel(r'$n_e \; [\lambda_D]$')
fig.tight_layout()

In [84]:
fig, ax = plt.subplots()
# piecewise_ds.sel(voltage=-4, probe='flat_flush')['delta'].plot(x='theta', ls='none', marker='x')
piecewise_ds.sel(voltage=-4, probe='flat_flush', theta=3)['pot'].plot.pcolormesh()

# $I$ plots

In [96]:
ff_probe = lpu.AngledTipProbe(a=5e-3, b=5e-3, L=5e-3, g=1e-3, d_perp=0e-4, 
                              theta_f=0.0, theta_p=0.0)

ar_probe = lpu.AngledTipProbe(a=5e-3, b=5e-3, L=5e-3, g=1e-3, d_perp=3e-4, 
                              theta_f=0.0, theta_p=np.radians(10.0))

In [100]:
# 3e-4
exposed_LE_angle = np.degrees(np.arctan(3e-4/1e-3))

In [124]:
c_s = lpu.sound_speed(T_e, gamma_i=2)
print(c_s)
I_0_bits =  (c.ELEM_CHARGE * n_0 * c_s)


37905.47429048637


In [102]:
fig, ax = plt.subplots()

dummy_theta_rads = np.radians(dummy_theta)

ax.axvline(x=exposed_LE_angle, **c.AX_LINE_DEFAULTS)

# ax.plot(dummy_theta, (L+g) * lambda_D * np.sin(dummy_theta_rads), label='ff_probe')
ax.plot(dummy_theta, ff_probe.get_2d_collection_length(dummy_theta_rads), label='ff_probe')
ax.plot(dummy_theta, ar_probe.get_2d_collection_length(dummy_theta_rads), label='ar_probe')

for j, probe in enumerate(probes):
    plot_ds = fflwp_shexp_ds.sel(probe=probe)
    print(plot_ds.recession.values)
    
    L_0 = get_L_0_p(L, g, dummy_theta_rads, plot_ds.theta_p_rads.values, plot_ds.recession.values/lambda_D) * lambda_D
    ax.plot(dummy_theta, L_0, label=f'L_0 {probe}')
    
ax.legend()

0.0
0.0003


In [139]:
fig, ax = plt.subplots()


ax.axvline(x=exposed_LE_angle, **c.AX_LINE_DEFAULTS)


ax.plot(dummy_theta, ff_probe.get_2d_collection_length(dummy_theta_rads), label='ff_probe')
ax.plot(dummy_theta, ar_probe.get_2d_collection_length(dummy_theta_rads), label='ar_probe')

for j, probe in enumerate(probes):
    plot_ds = fflwp_shexp_ds.sel(probe=probe)
    print(plot_ds.recession.values)
    
    L_0 = get_L_0_p(L, g, dummy_theta_rads, plot_ds.theta_p_rads.values, plot_ds.recession.values/lambda_D) * lambda_D
    ax.plot(dummy_theta, L_0, label=f'L_0 {probe}')
    
ax.legend()

0.0
0.0003


In [309]:
# Uncorrected Isat graph for either hg or wg (uncomment relevant preamble), probes split across axes

fig, ax = plt.subplots(1, 2, sharex=True, sharey='row', figsize=[10,6])

denormaliser = denormaliser_wg
datasets = [fwp0_shexp_ds, fflwp_shexp_ds]

probes = ['flat_flush', 'angled_recessed']


labels = [
    r'$V_w = 0$',
    r'$V_w = V_{fl}$',
]


for i, probe in enumerate(probes):
    for j, plot_ds in enumerate(datasets):
        if probe not in plot_ds.probe.values:
            continue
        plot_ds = plot_ds.sel(probe=probe, theta=slice(5,None))
        
        L_0 = get_L_0_rr(L, g, plot_ds.theta_rads, plot_ds.theta_p_rads.values, 
                         plot_ds.recession / lambda_D, 0.0) 
        
        uncorr_dens = get_density(plot_ds.ion_I_sat.values, L_0, denormaliser, c_s=c_s)
        measured_isat = denormaliser(plot_ds.ion_I_sat, 'current')
        calced_isat = probe_obj[i].get_isat(5, n_0, np.radians(dummy_theta)) / probe_obj[i].a
        calced_isat_coarse = probe_obj[i].get_isat(5, n_0, np.radians(plot_ds.theta_rads)) / probe_obj[i].a
        
        ax[i].plot(plot_ds.theta,  measured_isat, color=contrast_palette[j+2], 
                   label=labels[j], ls='none', marker=markers[j])
        ax[i].errorbar(dummy_theta, calced_isat, label=r'probe_def', fmt='k--', linewidth=0.8, alpha=0.6)

        
    
#     ax[i].axhline(y=n_0, **c.AX_LINE_DEFAULTS, label=r'$n_0$')
    ax[i].set_title(probe_names[probe])
    ax[i].legend()
    ax[i].legend()
    ax[i].set_xlim(0, 32.5)

    
ax[0].set_xlabel(r'$\theta \;[\degree]$')
ax[1].set_xlabel(r'$\theta \;[\degree]$')
ax[0].set_ylabel(r'$n_e \; [\lambda_D]$')

fig.tight_layout()

In [373]:
# Difference in A_coll graph for shexp, probes split across axes

fig, ax = plt.subplots(1, 2, sharex=True, sharey=False, figsize=[10,6])

denormaliser = denormaliser_wg
datasets = [fflwp_shexp_ds]


gamma_i = 2.0

probes = ['flat_flush', 'angled_recessed']
markers = ['o', '^', 's', 'p']

fit_type_label = 'ion'

isat_base = '_I_sat'
isat_label = fit_type_label + isat_base
d_isat_label = fit_type_label + '_d'+ isat_base

for i, probe in enumerate(probes):
    plot_ds = fflwp_shexp_ds.sel(probe=probe, theta=slice(5,None))
    
    measured_isat = denormaliser(plot_ds[isat_label], 'current')
    measured_d_isat = denormaliser(plot_ds[d_isat_label], 'current')
    calced_isat = probe_obj[i].get_isat(T_e, n_0, np.radians(dummy_theta), gamma_i=gamma_i) / probe_obj[i].a

    ax[0].errorbar(plot_ds.theta,  measured_isat, yerr=measured_d_isat, 
                   color=colours[i], mfc='none', label=probe_names[probe], 
                   ls='none', marker=markers[i])
    ax[0].errorbar(dummy_theta, calced_isat, label=f'{probe_names[probe]} - Expected', 
                   ls=linestyles[i+1], color=colours[i], linewidth=1.5)
    
    
    c_s = lpu.sound_speed(T_e, gamma_i=gamma_i)
    I_0_bits =  (c.ELEM_CHARGE * n_0 * c_s)

    a_coll = measured_isat / (I_0_bits * lambda_D)
    d_a_coll = measured_d_isat / (I_0_bits * lambda_D)

    calced_a_coll_coarse = probe_obj[i].get_2d_collection_length(plot_ds.theta_rads.values) / lambda_D
    ax[1].errorbar(plot_ds.theta, a_coll - calced_a_coll_coarse, yerr=d_a_coll, 
                   color=colours[i], mfc='none', ls='none', marker=markers[i], 
                   label=probe_names[probe])


    calced_a_coll = probe_obj[i].get_2d_collection_length(dummy_theta_rads) / lambda_D
#     ax[i].errorbar(dummy_theta, calced_a_coll, label=r'probe_def', fmt='k--', linewidth=0.8, alpha=0.6)
    
#     ax[1].set_title('Difference in ')
    
    ax[i].set_xlim(0, 32.5)

ax[0].set_ylim(0, 22.5)
    
ax[1].axhline(y=0, label=r'Expected', color='k', ls=':', linewidth=1.5, alpha=0.6)

    
ax[0].set_xlabel(r'$\theta \;[\degree]$')
ax[1].set_xlabel(r'$\theta \;[\degree]$')
ax[0].set_ylabel(r'$\hat{I}_{sat} \; $')
ax[1].set_ylabel(r'$\Delta L_0 \; [\lambda_D]$')

fig.tight_layout(rect=(0, 0, 1.0, 0.9))
ax[0].legend(ncol=2, loc='upper left', bbox_to_anchor=(0.18, 1.19))

In [311]:
# Uncorrected A_coll graph for either hg or wg (uncomment relevant preamble), probes split across axes

fig, ax = plt.subplots(2, 2, sharex=True, sharey='row', figsize=[10,6])

denormaliser = denormaliser_wg
datasets = [fflwp_shexp_ds]

gamma_is = [3/3, 2.0, 9/3]
gamma_i = 2.0

probes = ['flat_flush', 'angled_recessed']

markers = ['o', '^', 's', 'p']
labels = [
#     r'$V_w = 0$',
#     r'$V_w = V_{fl}$',
    r'$\gamma_i = 1$',
    r'$\gamma_i = 2$',
#     r'$\gamma_i = 5/3$',
    r'$\gamma_i = 3$',
]


for i, probe in enumerate(probes):
    plot_ds = fflwp_shexp_ds.sel(probe=probe, theta=slice(5,None))
    for j, gamma_i in enumerate(gamma_is):
        c_s = lpu.sound_speed(T_e, gamma_i=gamma_i)
        I_0_bits =  (c.ELEM_CHARGE * n_0 * c_s)

        dn_isat = denormaliser(plot_ds.ion_I_sat, 'current')
        a_coll = dn_isat / (I_0_bits * lambda_D)

        ax[0,i].plot(plot_ds.theta, a_coll, color=contrast_palette[j+1], 
                   label=labels[j], ls='none', marker=markers[j])

        calced_a_coll_coarse = probe_obj[i].get_2d_collection_length(plot_ds.theta_rads.values) / lambda_D
        ax[1,i].plot(plot_ds.theta, a_coll - calced_a_coll_coarse, color=contrast_palette[j+1], 
                     label=labels[j], ls='none', marker=markers[j])


    calced_a_coll = probe_obj[i].get_2d_collection_length(dummy_theta_rads) / lambda_D
    ax[0,i].errorbar(dummy_theta, calced_a_coll, label=r'probe_def', fmt='k--', linewidth=0.8, alpha=0.6)
    
    ax[0,i].set_title(probe_names[probe])
    ax[0,i].legend()
    ax[0,i].legend()
    ax[0,i].set_xlim(0, 32.5)
    
    ax[1,i].axhline(y=0, label=r'probe_def', color='k', ls='--', linewidth=0.8, alpha=0.6)

    
ax[1,0].set_xlabel(r'$\theta \;[\degree]$')
ax[1,1].set_xlabel(r'$\theta \;[\degree]$')
ax[0,0].set_ylabel(r'$L_0 \; [\lambda_D]$')
ax[1,0].set_ylabel(r'$\Delta L_0 \; [\lambda_D]$')



fig.tight_layout()


## Comparison of temperature fits

In [ ]:

probes = ['flat_flush','angled_recessed']

variable_str = '{}T_e'
fit_strs = ['str_iv_', 'expmt_iv_']
datasets = [hg_ds, hg_fflwp_ds]
# datasets = [fwp0_ds, fwp2_ds, fflwp_ds]
markers = ['o', '^', 's']
yax_labels = [
    r'$V_w = 0$',
    r'$V_w = 2$',
    r'$V_w = 4 \sim V_{fl}$',
]
leg_labels = ['4-Param (BMS)', '4-Param (Me)']

fig, axes = plt.subplots(len(fit_strs), len(probes), figsize=[12,12], 
                         sharey=True, sharex=True)

for j, probe in enumerate(probes):
    ax = axes[:,j]
    ax[0].set_title(probe)
    for i, plot_ds in enumerate(datasets):
        
        if probe in plot_ds.probe:
            plot_ds = plot_ds.sel(probe=probe)
        else:
            continue
        
        for k, fs in enumerate(fit_strs):
            ds_var = variable_str.format(fs)
            ds_err = variable_str.format(fs+'d_')
            ax[k].errorbar(plot_ds['theta'], plot_ds[ds_var], 
                           yerr=plot_ds[ds_err], marker=markers[i], 
                           mfc='none', linestyle='none', label=yax_labels[i], 
                           color=contrast_palette[i+1])
        
            if j == 0:
                ax[j].set_ylabel(f'{ds_var} [eV]')
            if k != 0:
                ax[j].set_xlabel(r'$\theta$ [$^{\circ}$]')

            ax[k].legend()
fig.tight_layout()

In [ ]:
fig, axes = plt.subplots(len(datasets), len(probes), figsize=[12,12], 
                         sharey='row', sharex=True)

for j, probe in enumerate(probes):
    ax = axes[:,j]
    ax[0].set_title(probe_names[probe])
    for k, fs in enumerate(fit_strs):
        for i, plot_ds in enumerate(datasets):
            if probe in plot_ds.probe:
                plot_ds = plot_ds.sel(probe=probe)
            else:
                continue
        
            ds_var = variable_str.format(fs)
            ds_err = variable_str.format(fs+'d_')
            ax[i].errorbar(plot_ds['theta'], plot_ds[ds_var], 
                           yerr=plot_ds[ds_err], marker=markers[k], 
                           mfc='none', linestyle='none', label=leg_labels[k], 
                           color=contrast_palette[k+2])
        
            if j == 0:
                ax[i].set_ylabel(r'$T_{e,p} / T_e$ [eV]')
                
            if i != len(datasets):
                ax[i].set_xlabel(r'$\theta$ [$^{\circ}$]')
            
            ax[i].legend(title=f'{yax_labels[i]}')
            ax[i].axhline(y=1.0, **c.AX_LINE_DEFAULTS)
fig.tight_layout()

In [ ]:
variable_str = '{}T_e'
fit_strs = ['str_iv_', 'expmt_iv_']
datasets = [fwp0_ds, fwp2_ds, fflwp_ds]
markers = ['o', '^', 's']
yax_labels = [
    r'$V_w = 0$',
    r'$V_w = 2$',
    r'$V_w = 4 \sim V_{fl}$',
]
leg_labels = ['4-Param (BMS)', '4-Param (Me)']

fig, axes = plt.subplots(len(datasets), len(probes), figsize=[12,12], 
                         sharey=True, sharex=True)

for j, probe in enumerate(probes):
    axes[0, j].set_title(probe)
    for k, fs in enumerate(fit_strs):
        for i, plot_ds in enumerate(datasets):
            ax = axes[k, j]
            if probe in plot_ds.probe:
                plot_ds = plot_ds.sel(probe=probe)
            else:
                continue
        
            ds_var = variable_str.format(fs)
            ds_err = variable_str.format(fs+'d_')
            ax[i].errorbar(plot_ds['theta'], plot_ds[ds_var], 
                           yerr=plot_ds[ds_err], marker=markers[k], 
                           mfc='none', linestyle='none', label=leg_labels[k], 
                           color=contrast_palette[k+1])
        
            if j == 0:
                ax[i].set_ylabel(f'{yax_labels[i]} [eV]')
            if k == 2:
                ax[k].set_xlabel(r'$\theta$ [$^{\circ}$]')

            ax[k].legend()
fig.tight_layout()

## Piecewise Analysis

In [ ]:
piecewise_ds

In [ ]:
fig, ax = plt.subplots()
piecewise_ds.sel(probe='flat_flush', theta=3.0).delta.plot(x='voltage')

In [ ]:
fig, ax = plt.subplots()

piecewise_ds.sel(probe='flat_flush', theta=3.0, voltage=-25.0).pot.plot.pcolormesh(cmap='viridis')

## Floating Potentials

In [115]:
fig, ax = plt.subplots()
fflwp_ds.sel(probe='flat_flush').str_iv_V_f.plot(x='theta', marker='.')

hg_fflwp_ds.sel(probe='flat_flush').str_iv_V_f.plot(x='theta', marker='.')
# burst['n'].plot(x='radial', y='binormal', col='time', col_wrap=2, aspect=0.8/0.35)

In [118]:
# fig, ax = plt.subplots()
# fig = plt.figure()
fflwp_shexp_ds.sel(probe='flat_flush', theta=[4,6,8,10,20,30])['voltage_wall'].plot(x='voltage', col='theta', col_wrap=3)
# hg_fflwp_ds.sel(probe='flat_flush', theta=[3,5,8,10,20,30])['voltage_wall'].plot(x='voltage', col='theta', col_wrap=3)

In [121]:
fig, ax = plt.subplots(1, 2)

datasets = [fflwp_shexp_ds, fwp0_shexp_ds]
for plot_ds in datasets:
    for i, probe in enumerate(probes):
        ds = plot_ds.sel(probe=probe)
        ds.str_iv_V_f.plot(x='theta', marker='.', ax=ax[i])
        ds['voltage_wall'].mean('voltage').plot(x='theta', ax=ax[i])


## Probe diagrams

In [52]:
import flopter.spice.utils as spu

In [53]:
importlib.reload(spu)

<module 'flopter.spice.utils' from '/home/jleland/coding/projects/flopter/flopter/spice/utils.py'>

In [54]:
probe_inputs = []
for i in [3,2]:
    spl_path = spice_dir / all_run_dirs[scans[i]][0]
    probe_inputs.append(next(next(spl_path.glob('backup*')).glob('input.inp')))
    
# probe_inputs

fig = plt.figure(figsize=[10,4])
gs = fig.add_gridspec(ncols=175, nrows=1)

ax_ff = fig.add_subplot(gs[0, :70])
ax_ar = fig.add_subplot(gs[0, 75:], sharey=ax_ff)



# # Hide x tick-labels
# for axis in [ax_s, ax_v, ax_s_ave, ax_v_ave]:
#     plt.setp(axis.get_xticklabels(), visible=False)
# Hide y tick-labels
for axis in [ax_ar]:
    plt.setp(axis.get_yticklabels(), visible=False)

axes = np.array(
    [ax_ff, ax_ar])


# fig, axes = plt.subplots(1, 2, figsize=[10,4], sharex=False, sharey=True)
colours = [cb_palette[0], cb_palette[4]]

for i, probe_inp in enumerate(probe_inputs):
    ax = axes[i]
    spu.plot_2d_sim_window(probe_inp, ax=ax, colour=colours[i])
    ax.set_ylim(0,232)
    
axes[1].set_xlim(0,922) #759)
axes[0].set_xlim(0,645.4) #759)
axes[1].set_ylabel('')

    
fig.tight_layout()

{'triangle': [], 'rectangle': [<Section: rectangle0>, <Section: rectangle1>, <Section: rectangle2>], 'circle': []}
{'triangle': [<Section: triangle0>], 'rectangle': [<Section: rectangle0>, <Section: rectangle1>, <Section: rectangle2>], 'circle': []}


/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:40: UserWarning: Tight layout not applied. tight_layout cannot make axes width small enough to accommodate all axes decorations
